In [ ]:
# ! pip install --upgrade pip
# ! pip install --user numpy scipy matplotlib seaborn boost_histogram pandas uproot awkward-pandas uproot3

In [ ]:
# import uproot3 as uproot
import uproot
import numpy as np

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# plt.style.use("belle2")

import seaborn as sns

import sys

# import plothist
# plt.style.use("belle2")
# from plothist import make_2d_hist, plot_2d_hist_with_projections

In [ ]:
plt.rcParams.update({
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.titlesize": 16
})

In [ ]:
sys.path.append("/home/belle2/amubarak/Ds2D0enue_Analysis/10-Python_Functions/")

# Prep-Work

### Import Data

In [ ]:
# In this notebook we only process the main signal and the generic events,
# for illustration purposes.
# You can add other backgrounds after if you wish.
samples = ["Signal","ccbar","charged","ddbar","mixed","ssbar","uubar"]

DataFrames = {}  # define empty dictionary to hold dataframes
Date = "0506"
Attempt = "0"

# Signal:
DataFrames[samples[0]] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:Dstree",library='pd')
# Background
for s in samples[1:]: # loop over samples
    DataFrames[s] =  uproot.concatenate("/group/belle2/users2022/amubarak/02-Grid/Sample_KEKCC/Ds2D0e-Generic_Ds_" + Date +"25_"+ Attempt +"_"+ s +".root:Dstree",library='pd')

# Define background samples
background_samples = ["ccbar","charged","ddbar","mixed","ssbar","uubar"]

# Concatenate all background DataFrames into one called "All"
DataFrames["All"] = pd.concat([DataFrames[s] for s in background_samples], ignore_index=True)

# Combine light-quark backgrounds into a single 'uds' category
DataFrames["uds"] = pd.concat(
    [DataFrames["uubar"], DataFrames["ddbar"], DataFrames["ssbar"]],
    ignore_index=True
)

In [ ]:
# # In this notebook we only process the main signal and the generic events,
# # for illustration purposes.
# # You can add other backgrounds after if you wish.
# samples = ["Signal","BB","ccbar","ddbar","ssbar","taupair","uubar"]

# DataFrames = {}  # define empty dictionary to hold dataframes
# Date = "0419"
# Attempt = "0"

# # Signal:
# DataFrames[samples[0]] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:Dstree",library='pd')
# # Background
# for s in samples[1:]: # loop over samples
#     DataFrames[s] =  uproot.concatenate("/group/belle2/users2022/amubarak/02-Grid/Sample_Grid/Ds2D0e-Generic_Ds_" + Date +"25_"+ Attempt +"_"+ s +".root:Dstree",library='pd')

# # Define background samples
# background_samples = ["BB", "ccbar", "ddbar", "ssbar", "taupair", "uubar"]

# # Concatenate all background DataFrames into one called "All"
# DataFrames["All"] = pd.concat([DataFrames[s] for s in background_samples], ignore_index=True)

# # Combine light-quark backgrounds into a single 'uds' category
# DataFrames["uds"] = pd.concat(
#     [DataFrames["uubar"], DataFrames["ddbar"], DataFrames["ssbar"]],
#     ignore_index=True
# )

In [ ]:
pd.set_option('display.max_rows', 200000)
pd.set_option('display.max_columns', 200000)

The line below is to look at the available variables.

In [ ]:
DataFrames["uds"].columns.tolist()

### Setup
The code below will be used to apply cuts to the data.  
The range of the plots.

In [ ]:
# Electron ID
#-------------------
# DataFrames[samples[i]] = DataFrames[samples[i]][DataFrames[samples[i]]['e_electronID']>=0.95]
# DataFrames[samples[i]] = DataFrames[samples[i]][DataFrames[samples[i]]['e_electronID']>=0.95]
# DataFrames[samples[i]] = DataFrames[samples[i]][DataFrames[samples[i]]['Ds_gammaveto_em_electronID']>=0.95]
# DataFrames[samples[i]] = DataFrames[samples[i]][DataFrames[samples[i]]['Ds_gammaveto_em_electronID']>=0.95]

# Fake D0 Suppression
#------------------------
# DataFrames[samples[i]] = DataFrames[samples[i]][DataFrames[samples[i]]['Ds_extraInfo_FastBDT']>=0.58]
# DataFrames[samples[i]] = DataFrames[samples[i]][DataFrames[samples[i]]['Ds_extraInfo_FastBDT']>=0.58]

# Fake D0 Suppression
# ------------------------
# for i in range(len(samples)):
#     DataFrames[samples[i]] = DataFrames[samples[i]][DataFrames[samples[i]]["Ds_extraInfo_BkgBDT"]>=0.531]
# DataFrames[samples[i]] = DataFrames[samples[i]][DataFrames[samples[i]]["Ds_extraInfo_BkgBDT"]>=0.531]

# Peaking Background Removal
#----------------------------
# DataFrames[samples[i]] = DataFrames[samples[i]][(DataFrames[samples[i]]['Ds_diff_D0pi']>=0.15)]
# DataFrames[samples[i]] = DataFrames[samples[i]][(DataFrames[samples[i]]['Ds_diff_D0pi']>=0.15)]

# Photon Conversion
#-------------------
# DataFrames[samples[i]] = DataFrames[samples[i]][DataFrames[samples[i]]['Ds_gammaveto_M_Correction']<=0.1]
# DataFrames[samples[i]] = DataFrames[samples[i]][DataFrames[samples[i]]['Ds_gammaveto_M_Correction']<=0.1]

# # Vertex Fitting
# #----------------
# DataFrames[samples[i]] = DataFrames[samples[i]][DataFrames[samples[i]]['Ds_chiProb']>=0.01]
# DataFrames[samples[i]] = DataFrames[samples[i]][DataFrames[samples[i]]['Ds_chiProb']>=0.01]

# Vertex Fit
#----------------
# DataFrames[samples[i]] = DataFrames[samples[i]][DataFrames[samples[i]]['Ds_chiProb_Ds_rank']==1]
# DataFrames[samples[i]] = DataFrames[samples[i]][DataFrames[samples[i]]['Ds_chiProb_Ds_rank']==1]

In [ ]:
import numpy as np

df = DataFrames["uds"]

# Step 1: Build 3-momentum arrays
p_D0 = df[["D0_px", "D0_py", "D0_pz"]].to_numpy()
p_e  = df[["e_px", "e_py", "e_pz"]].to_numpy()

# Step 2: Energy calculator
def compute_energy(p, m):
    return np.sqrt(np.sum(p**2, axis=1) + m**2)

# Step 3: Delta m calculator
def compute_delta_m(p_D0, p_e, m_D0_array, m_e_hyp):
    E_D0 = compute_energy(p_D0, m_D0_array)
    E_e  = compute_energy(p_e, m_e_hyp)

    p_Ds = p_D0 + p_e
    E_Ds = E_D0 + E_e

    p_Ds2 = np.sum(p_Ds**2, axis=1)
    m_Ds = np.sqrt(E_Ds**2 - p_Ds2)
    return m_Ds - m_D0_array

# Step 4: Mass hypotheses
mass_hypotheses = {
    "electron": 0.000511,
    "pion":     0.1396,
    "kaon":     0.4937,
    "muon":     0.1057,
    "proton":   0.9383
}

# Step 5: Compute Δm for each hypothesis
m_D0_array = df["D0_M"].to_numpy()

for label, m_e in mass_hypotheses.items():
    df[f"DeltaM_{label}"] = compute_delta_m(p_D0, p_e, m_D0_array, m_e)

In [ ]:
DataFrames["All"].isna().sum()

In [ ]:
print(abs(DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames["All"]['D0_isSignal'])==1)][['Ds_mcPDG']]).value_counts(dropna=False))
print(abs(DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Dstar0'])==1) & (abs(DataFrames["All"]['D0_isSignal'])==1)][['Ds_mcPDG']]).value_counts(dropna=False))
print(abs(DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames["All"]['D0_isSignal'])==1)][['D0_mcPDG']]).value_counts(dropna=False))
print(abs(DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Other'])==1) | ((abs(DataFrames["All"]['D0_mcPDG'])==421) & (abs(DataFrames["All"]['D0_isSignal'])==0))][['K_genMotherPDG']]).value_counts(dropna=False))

In [ ]:
print(len(DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames["All"]['D0_isSignal'])==1)]))
print(len(DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Dstar0'])==1) & (abs(DataFrames["All"]['D0_isSignal'])==1)]))
print(len(DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames["All"]['D0_isSignal'])==1)]))
print(len(DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Other'])==1) | ((abs(DataFrames["All"]['D0_mcPDG'])==421) & (abs(DataFrames["All"]['D0_isSignal'])==0))]))

# Single Variable  
The code below is to look at any specific variable closely.

$D^{*+}$

In [ ]:
dM = -1
FD = -1
BD = -1

DstarPlus_Truth = DataFrames["All"][(abs(DataFrames["All"]['D0_genMotherPDG'])==413) & (abs(DataFrames["All"]['D0_isSignal'])==1)]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Settings ===
Stacked = False
Density = False
Bins = 50
var = 'Ds_diff_D0pi'
Range = [0.1, 0.55]
BS = -1
dM = 0.06
eID = 0.1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# === Background Histogram (DstarPlus_Truth) ===
labels1 = [r'$D^{*+} \rightarrow [D^{0} \rightarrow K^{-} \pi^{+}] \pi^{+}$']
colors1 = ['C2']
data1 = [
    DstarPlus_Truth[
        (DstarPlus_Truth['Ds_extraInfo_BkgBDT'] >= BS) &
        (DstarPlus_Truth['Ds_gammaveto_M_Correction'] <= dM) &
        (DstarPlus_Truth['Ds_gammaveto_em_electronID'] >= eID)
    ][var]
]
scales1 = [5.0]  # <-- You can change this to whatever factor you want

plt.hist(
    data1,
    color=colors1,
    label=labels1,
    density=Density,
    stacked=Stacked,
    bins=Bins,
    alpha=1,
    histtype='step',
    linewidth=2,
    range=Range,
    weights=[np.ones_like(d) * s for d, s in zip(data1, scales1)]
)

# === Signal Histograms (from signal MC) ===
labels2 = [r'$isSignal(D_s^{+})=1$ (signal MC)', r'$isSignal(D_s^{+})=0$ (signal MC)']
colors2 = ["#4C6EB1", 'C3']
data2 = [
    DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal'] == 1)][var],
    DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal'] == 0)][var]
]
scales2 = [1.0, 8.0]  # <-- independent scaling for signal MC categories

plt.hist(
    data2[::-1],
    color=colors2[::-1],
    label=labels2[::-1],
    alpha=1,
    range=Range,
    linewidth=2,
    stacked=Stacked,
    density=Density,
    bins=Bins,
    histtype='step',
    weights=[np.ones_like(d) * s for d, s in zip(data2[::-1], scales2[::-1])]
)

plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
plt.axvline(0.16,ls='--',color='gray')
plt.axvline(0.28,ls='--',color='gray')
plt.axvspan(0.28,Range[1],color='gray',alpha=0.2)

# === Titles and labels ===
# plt.title(r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$', loc="left")
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width=perBin))
plt.xlabel(r'$\Delta m_{\pi}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Settings ===
Stacked = False
Density = True
Bins = 50
var = 'e_p'  # now plotting electron momentum
cut_var = 'Ds_diff_D0pi'
cut_min, cut_max = 0.16, 0.28  # non-shaded region
BS = -1
dM = 0.06
eID = 0.1
Samples = "All"
Range = [0.0, 0.6]  # Adjusted for e_p
perBin = ((Range[1] - Range[0]) / Bins) * 1000
print("Width Per Bin: {:.2f} MeV".format(perBin))

# === Background Histogram (DstarPlus_Truth) ===
labels1 = [r'$D^{*+} \rightarrow [D^{0} \rightarrow K^{-} \pi^{+}] \pi^{+}$']
colors1 = ['C2']
data1 = [
    DstarPlus_Truth[
        (DstarPlus_Truth[cut_var] > cut_min) &
        (DstarPlus_Truth[cut_var] < cut_max) &
        (DstarPlus_Truth['Ds_extraInfo_BkgBDT'] >= BS) &
        (DstarPlus_Truth['Ds_gammaveto_M_Correction'] <= dM) &
        (DstarPlus_Truth['Ds_gammaveto_em_electronID'] >= eID)
    ][var]
]
scales1 = [5.0]

plt.hist(
    data1,
    color=colors1,
    label=labels1,
    density=Density,
    stacked=Stacked,
    bins=Bins,
    alpha=1,
    histtype='step',
    linewidth=2,
    range=Range,
    weights=[np.ones_like(d) * s for d, s in zip(data1, scales1)]
)

# === Signal Histograms (from signal MC) ===
labels2 = [r'$isSignal(D_s^{+})=1$ (signal MC)', r'$isSignal(D_s^{+})=0$ (signal MC)']
colors2 = ["#4C6EB1", 'C3']
data2 = [
    DataFrames["Signal"][
        (DataFrames["Signal"][cut_var] > cut_min) &
        (DataFrames["Signal"][cut_var] < cut_max) &
        (DataFrames["Signal"]['Ds_isSignal'] == 1)
    ][var],
    
    DataFrames["Signal"][
        (DataFrames["Signal"][cut_var] > cut_min) &
        (DataFrames["Signal"][cut_var] < cut_max) &
        (DataFrames["Signal"]['Ds_isSignal'] == 0)
    ][var]
]
scales2 = [1.0, 8.0]

plt.hist(
    data2[::-1],
    color=colors2[::-1],
    label=labels2[::-1],
    alpha=1,
    range=Range,
    linewidth=2,
    stacked=Stacked,
    density=Density,
    bins=Bins,
    histtype='step',
    weights=[np.ones_like(d) * s for d, s in zip(data2[::-1], scales2[::-1])]
)

# === Labels ===
plt.title(r'$0.16 \leq \Delta m(D_s^{+} - D^0) \leq 0.28$', loc="left")
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c)$'.format(width=perBin))
plt.xlabel(r'$p_{lab}(e^{+})$ [GeV/$c$]')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Settings ===
Stacked = False
Density = False
Bins = 50
var = 'Ds_daughterAngle_0_1'  # angle between e+ and D0
Range = [0, np.pi]
BS = -1
dM = 0.06
eID = 0.1
Samples = "All"
peak_min, peak_max = 0.16, 0.28
perBin = ((Range[1] - Range[0]) / Bins)
print("Width Per Bin: {width:.2f} radians".format(width=perBin))

# === Background Histogram (DstarPlus_Truth) in Δm peak region ===
labels1 = [r'$D^{*+} \rightarrow [D^{0} \rightarrow K^{-} \pi^{+}] \pi^{+}$']
colors1 = ['C2']
data1 = [
    DstarPlus_Truth[
        (DstarPlus_Truth['Ds_diff_D0pi'] > peak_min) &
        (DstarPlus_Truth['Ds_diff_D0pi'] < peak_max) &
        (DstarPlus_Truth['Ds_extraInfo_BkgBDT'] >= BS) &
        (DstarPlus_Truth['Ds_gammaveto_M_Correction'] <= dM) &
        (DstarPlus_Truth['Ds_gammaveto_em_electronID'] >= eID)
    ][var]
]
scales1 = [5.0]

plt.hist(
    data1,
    color=colors1,
    label=labels1,
    density=Density,
    stacked=Stacked,
    bins=Bins,
    alpha=1,
    histtype='step',
    linewidth=2,
    range=Range,
    weights=[np.ones_like(d) * s for d, s in zip(data1, scales1)]
)

# === Signal Histograms (from signal MC) over full Δm ===
labels2 = [r'$isSignal(D_s^{+})=1$ (signal MC)', r'$isSignal(D_s^{+})=0$ (signal MC)']
colors2 = ["#4C6EB1", 'C3']
data2 = [
    DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal'] == 1)][var],
    DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal'] == 0)][var]
]
scales2 = [1.0, 8.0]

plt.hist(
    data2[::-1],
    color=colors2[::-1],
    label=labels2[::-1],
    alpha=1,
    range=Range,
    linewidth=2,
    stacked=Stacked,
    density=Density,
    bins=Bins,
    histtype='step',
    weights=[np.ones_like(d) * s for d, s in zip(data2[::-1], scales2[::-1])]
)

# === Titles and labels ===
plt.title(r'$0.16 \leq \Delta m(D_s^{+} - D^0) \leq 0.28$', loc="left")
plt.ylabel(r'$Entries/(\; {width:.2f}\;rad)$'.format(width=perBin))
plt.xlabel(r'$\theta(e^+, D^0)$ [rad]')
plt.legend()
plt.show()


In [ ]:
# Stacked = False
# Density = False
# # Bins = 50
# # var = 'Ds_diff_D0pi'
# var = 'DeltaM_proton'
# # var = 'Ds_massDifference_0'
# Range = [0.8, 2.2]
# dM = 0.06
# FD = -1
# BS = -1
# eID = 0.1
# Samples = "All"
# perBin = ((Range[1] - Range[0])/Bins)*1000
# print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# label1= r'$BDT \geq 0.531, \; Real \; Electrons$'
# label2= r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$'

# labels=[label1,label2]
# colors=['C2','C3']
# data=[
#       DstarPlus_Truth[(abs(DstarPlus_Truth['e_mcPDG'])==11) & (DstarPlus_Truth['Ds_extraInfo_BkgBDT']>=0.531) & (DstarPlus_Truth['Ds_gammaveto_M_Correction']>=-1) & (DstarPlus_Truth['Ds_gammaveto_em_electronID']>=-1)][var],
#       DstarPlus_Truth[(DstarPlus_Truth['Ds_extraInfo_BkgBDT']>=BS) & (DstarPlus_Truth['Ds_gammaveto_M_Correction']<=dM) & (DstarPlus_Truth['Ds_gammaveto_em_electronID']>=eID)][var],
#       ]

# scaling_factors = [3.0, 1.0]  # Scale first histogram normally, second one is halved
# weights = [
#     scaling_factors[0] * np.ones_like(data[0]),
#     scaling_factors[1] * np.ones_like(data[1])
# ]
# plt.hist(
#     data,
#     weights=weights,
#     color=colors,
#     label=labels,
#     density=Density,
#     stacked=Stacked,
#     bins=Bins,
#     alpha=1,
#     histtype='step',
#     linewidth=2,
#     range=Range
# )
# # plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# # plt.axvline(0.16,ls='--',color='gray')
# # plt.axvline(0.28,ls='--',color='gray')
# # plt.axvspan(0.28,Range[1],color='gray',alpha=0.2)

# # Title
# #--------
# plt.title(r'$D^{*+} \rightarrow [D^{0} \rightarrow K^{-} \pi^{+}] \pi^{+}$', loc = "left")
# plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# # Label
# #-------
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$\Delta m_{\pi}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# # plt.yscale("log")
# # plt.xscale("log")
# # plt.ylim(0, 4)
# plt.legend()
# plt.show()

In [ ]:
Stacked = False
Density = False
# Bins = 50
var = 'Ds_diff_D0pi'
# var = 'Ds_massDifference_0'
Range = [0.1, 0.55]
dM = 0.06
FD = -1
BS = -1
eID = -1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1 = r'$BDT \geq 0.531$, Truth-matched $e^-$'

labels=[label1]
colors=['C2']
data=[
      DstarPlus_Truth[(abs(DstarPlus_Truth['e_mcPDG'])==11) & (DstarPlus_Truth['Ds_extraInfo_BkgBDT']>=0.531) & (DstarPlus_Truth['Ds_gammaveto_M_Correction']>=-1) & (DstarPlus_Truth['Ds_gammaveto_em_electronID']>=-1)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=2, range=Range)
plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
plt.axvline(0.16,ls='--',color='gray')
plt.axvline(0.28,ls='--',color='gray')
plt.axvspan(0.28,Range[1],color='gray',alpha=0.2)

# Title
#--------
plt.title(r'$D^{*+} \rightarrow [D^{0} \rightarrow K^{-} \pi^{+}] \pi^{+}$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m_{\pi}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 6)
plt.legend()
plt.show()

$D^{*0}$

In [ ]:
dM = -1
FD = -1
BD = -1

Dstar0_Truth = DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Dstar0'])==1) & (abs(DataFrames["All"]['D0_isSignal'])==1)]

In [ ]:
Stacked = False
Density = False
Bins = 50
# var = 'e_pt'
# var = 'Ds_diff_D0pi'
var = 'Ds_massDifference_0'
Range = [0.0, 0.25]
BS = -1
dM = 0.06
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))


labels2=[r'$D^{*0} \; (Comb.)$',r'$D^{*0} \; (Peak)$']
colors2=['C1','C3']
data2=[
      Dstar0_Truth[(~abs(Dstar0_Truth["Ds_mcPDG"]).isna()) & (abs(Dstar0_Truth["Ds_mcPDG"])!=423) & (Dstar0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (Dstar0_Truth['Ds_gammaveto_M_Correction']<=0.06) & (Dstar0_Truth['Ds_gammaveto_em_electronID']>=0.1)][var],
      Dstar0_Truth[(abs(Dstar0_Truth["Ds_mcPDG"])==423) & (Dstar0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (Dstar0_Truth['Ds_gammaveto_M_Correction']<=0.06) & (Dstar0_Truth['Ds_gammaveto_em_electronID']>=0.1)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data2, color=colors2, label=labels2, density=Density, stacked=True, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# plt.axvline(0.16,ls='--',color='gray')

# Title
#--------
plt.title(r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m_{e}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 6000)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
var = 'Ds_diff_D0pi'
# var = 'Ds_massDifference_0'
Range = [0.1, 0.55]
BS = -1
dM = 0.06
eID = 0.1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$D^{*0} \rightarrow [D^{0} \rightarrow K^{-} \pi^{+}] \gamma, [D^{0} \rightarrow K^{-} \pi^{+}] \pi^{0}$'

labels=[label1]
colors=['C1']
data=[
      Dstar0_Truth[(Dstar0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (Dstar0_Truth['Ds_gammaveto_M_Correction']<=dM) & (Dstar0_Truth['Ds_gammaveto_em_electronID']>=eID)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=2, range=Range)
# plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# plt.axvline(0.16,ls='--',color='gray')

# Title
#--------
plt.title(r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m_{\pi}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.xlabel(r'$p_{t}(e)\;[GeV/c]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 6000)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = True
# Bins = 50
# var = 'Ds_diff_D0pi'
var = 'Ds_massDifference_0'
Range = [0.0, 0.25]
dM = 0.06
FD = -1
BS = -1
eID = 0.1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$BDT \geq 0.531$'
label2= r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$'

labels=[label1,label2]
colors=['C1','C3']
data=[
      Dstar0_Truth[(Dstar0_Truth['Ds_extraInfo_BkgBDT']>=0.531) & (Dstar0_Truth['Ds_gammaveto_M_Correction']>=-1) & (Dstar0_Truth['Ds_gammaveto_em_electronID']>=eID)][var],
      Dstar0_Truth[(Dstar0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (Dstar0_Truth['Ds_gammaveto_M_Correction']<=dM) & (Dstar0_Truth['Ds_gammaveto_em_electronID']>=eID)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=2, range=Range)
# plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# plt.axvline(0.16,ls='--',color='gray')
# plt.axvline(0.26,ls='--',color='gray')
# plt.axvspan(0.26,Range[1],color='gray',alpha=0.2)

# Title
#--------
plt.title(r'$D^{*0} \rightarrow [D^{0} \rightarrow K^{-} \pi^{+}] \gamma, [D^{0} \rightarrow K^{-} \pi^{+}] \pi^{0}$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m_{e}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 4000)
plt.legend()
plt.show()

$D^{0}$

In [ ]:
dM = -1
FD = -1
BD = -1

D0_Truth = DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames["All"]['D0_isSignal'])==1)]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Settings ===
Stacked = False
Density = False
Bins = 50
var = 'Ds_diff_D0pi'
Range = [0.1, 0.55]
BS = -1
dM = 0.06
eID = 0.1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# === Background Histogram (D0_Truth) ===
labels1 = [r'$D^{0} \rightarrow K^{-} \pi^{+}$']
colors1 = ['C4']
data1 = [
    D0_Truth[
        (D0_Truth['Ds_extraInfo_BkgBDT'] >= BS) &
        (D0_Truth['Ds_gammaveto_M_Correction'] <= dM) &
        (D0_Truth['Ds_gammaveto_em_electronID'] >= eID)
    ][var]
]
scales1 = [7.0]  # <-- You can change this to whatever factor you want

plt.hist(
    data1,
    color=colors1,
    label=labels1,
    density=Density,
    stacked=Stacked,
    bins=Bins,
    alpha=1,
    histtype='step',
    linewidth=2,
    range=Range,
    weights=[np.ones_like(d) * s for d, s in zip(data1, scales1)]
)

# === Signal Histograms (from signal MC) ===
labels2 = [r'$isSignal(D_s^{+})=1$ (signal MC)', r'$isSignal(D_s^{+})=0$ (signal MC)']
colors2 = ["#4C6EB1", 'C3']
data2 = [
    DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal'] == 1)][var],
    DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal'] == 0)][var]
]
scales2 = [1.0, 8.0]  # <-- independent scaling for signal MC categories

plt.hist(
    data2[::-1],
    color=colors2[::-1],
    label=labels2[::-1],
    alpha=1,
    range=Range,
    linewidth=2,
    stacked=Stacked,
    density=Density,
    bins=Bins,
    histtype='step',
    weights=[np.ones_like(d) * s for d, s in zip(data2[::-1], scales2[::-1])]
)

plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
plt.axvline(0.16,ls='--',color='gray')
plt.axvline(0.28,ls='--',color='gray')
plt.axvspan(0.28,Range[1],color='gray',alpha=0.2)

# === Titles and labels ===
# plt.title(r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$', loc="left")
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width=perBin))
plt.xlabel(r'$\Delta m_{\pi}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Settings ===
Stacked = False
Density = True
Bins = 50
var = 'e_p'  # now plotting electron momentum
cut_var = 'Ds_diff_D0pi'
cut_min, cut_max = 0.16, 0.28  # non-shaded region
BS = -1
dM = 0.06
eID = 0.1
Samples = "All"
Range = [0.0, 0.6]  # Adjusted for e_p
perBin = ((Range[1] - Range[0]) / Bins) * 1000
print("Width Per Bin: {:.2f} MeV".format(perBin))

# === Background Histogram (D0_Truth) ===
labels1 = [r'$D^{0} \rightarrow K^{-} \pi^{+}$']
colors1 = ['C4']
data1 = [
    D0_Truth[
        (D0_Truth[cut_var] > cut_min) &
        (D0_Truth[cut_var] < cut_max) &
        (D0_Truth['Ds_extraInfo_BkgBDT'] >= BS) &
        (D0_Truth['Ds_gammaveto_M_Correction'] <= dM) &
        (D0_Truth['Ds_gammaveto_em_electronID'] >= eID)
    ][var]
]
scales1 = [5.0]

plt.hist(
    data1,
    color=colors1,
    label=labels1,
    density=Density,
    stacked=Stacked,
    bins=Bins,
    alpha=1,
    histtype='step',
    linewidth=2,
    range=Range,
    weights=[np.ones_like(d) * s for d, s in zip(data1, scales1)]
)

# === Signal Histograms (from signal MC) ===
labels2 = [r'$isSignal(D_s^{+})=1$ (signal MC)', r'$isSignal(D_s^{+})=0$ (signal MC)']
colors2 = ["#4C6EB1", 'C3']
data2 = [
    DataFrames["Signal"][
        (DataFrames["Signal"][cut_var] > cut_min) &
        (DataFrames["Signal"][cut_var] < cut_max) &
        (DataFrames["Signal"]['Ds_isSignal'] == 1)
    ][var],
    
    DataFrames["Signal"][
        (DataFrames["Signal"][cut_var] > cut_min) &
        (DataFrames["Signal"][cut_var] < cut_max) &
        (DataFrames["Signal"]['Ds_isSignal'] == 0)
    ][var]
]
scales2 = [1.0, 8.0]

plt.hist(
    data2[::-1],
    color=colors2[::-1],
    label=labels2[::-1],
    alpha=1,
    range=Range,
    linewidth=2,
    stacked=Stacked,
    density=Density,
    bins=Bins,
    histtype='step',
    weights=[np.ones_like(d) * s for d, s in zip(data2[::-1], scales2[::-1])]
)

# === Labels ===
plt.title(r'$0.16 \leq \Delta m(D_s^{+} - D^0) \leq 0.28$', loc="left")
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c)$'.format(width=perBin))
plt.xlabel(r'$p_{lab}(e^{+})$ [GeV/$c$]')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Settings ===
Stacked = False
Density = False
Bins = 50
var = 'Ds_daughterAngle_0_1'  # angle between e+ and D0
Range = [0, np.pi]
BS = -1
dM = 0.06
eID = 0.1
Samples = "All"
peak_min, peak_max = 0.16, 0.28
perBin = ((Range[1] - Range[0]) / Bins)
print("Width Per Bin: {width:.2f} radians".format(width=perBin))

# === Background Histogram (D0_Truth) in Δm peak region ===
labels1 = [r'$D^{0} \rightarrow K^{-} \pi^{+}$']
colors1 = ['C4']
data1 = [
    D0_Truth[
        (D0_Truth['Ds_diff_D0pi'] > peak_min) &
        (D0_Truth['Ds_diff_D0pi'] < peak_max) &
        (D0_Truth['Ds_extraInfo_BkgBDT'] >= BS) &
        (D0_Truth['Ds_gammaveto_M_Correction'] <= dM) &
        (D0_Truth['Ds_gammaveto_em_electronID'] >= eID)
    ][var]
]
scales1 = [5.0]

plt.hist(
    data1,
    color=colors1,
    label=labels1,
    density=Density,
    stacked=Stacked,
    bins=Bins,
    alpha=1,
    histtype='step',
    linewidth=2,
    range=Range,
    weights=[np.ones_like(d) * s for d, s in zip(data1, scales1)]
)

# === Signal Histograms (from signal MC) over full Δm ===
labels2 = [r'$isSignal(D_s^{+})=1$ (signal MC)', r'$isSignal(D_s^{+})=0$ (signal MC)']
colors2 = ["#4C6EB1", 'C3']
data2 = [
    DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal'] == 1)][var],
    DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal'] == 0)][var]
]
scales2 = [1.0, 8.0]

plt.hist(
    data2[::-1],
    color=colors2[::-1],
    label=labels2[::-1],
    alpha=1,
    range=Range,
    linewidth=2,
    stacked=Stacked,
    density=Density,
    bins=Bins,
    histtype='step',
    weights=[np.ones_like(d) * s for d, s in zip(data2[::-1], scales2[::-1])]
)

# === Titles and labels ===
plt.title(r'$0.16 \leq \Delta m(D_s^{+} - D^0) \leq 0.28$', loc="left")
plt.ylabel(r'$Entries/(\; {width:.2f}\;rad)$'.format(width=perBin))
plt.xlabel(r'$\theta(e^+, D^0)$ [rad]')
plt.legend()
plt.show()


In [ ]:
Stacked = False
Density = False
# Bins = 50
var = 'Ds_diff_D0pi'
# var = 'Ds_massDifference_0'
Range = [0.1, 0.55]
dM = 0.06
FD = -1
BS = -1
eID = 0.1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$BDT \geq 0.531, \; Real \; Electrons$'
label2= r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$'

labels=[label1,label2]
colors=['C4','C3']
data=[
      D0_Truth[(abs(D0_Truth['e_mcPDG'])==11) & (D0_Truth['Ds_extraInfo_BkgBDT']>=0.531) & (D0_Truth['Ds_gammaveto_M_Correction']>=-1) & (D0_Truth['Ds_gammaveto_em_electronID']>=-1)][var],
      D0_Truth[(D0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (D0_Truth['Ds_gammaveto_M_Correction']<=dM) & (D0_Truth['Ds_gammaveto_em_electronID']>=eID)][var],
      ]

scaling_factors = [3.5, 1.0]  # Scale first histogram normally, second one is halved
weights = [
    scaling_factors[0] * np.ones_like(data[0]),
    scaling_factors[1] * np.ones_like(data[1])
]
plt.hist(
    data,
    weights=weights,
    color=colors,
    label=labels,
    density=Density,
    stacked=Stacked,
    bins=Bins,
    alpha=1,
    histtype='step',
    linewidth=2,
    range=Range
)
plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
plt.axvline(0.16,ls='--',color='gray')
plt.axvline(0.28,ls='--',color='gray')
plt.axvspan(0.28,Range[1],color='gray',alpha=0.2)

# Title
#--------
plt.title(r'$D^{0} \rightarrow K^{-} \pi^{+}$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m_{\pi}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 4)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
# Bins = 50
# var = 'Ds_diff_D0pi'
var = 'Ds_massDifference_0'
Range = [0.0, 0.25]
dM = 0.06
FD = -1
BS = -1
eID = 0.1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1 = r'$BDT \geq 0.531$, Truth-matched $e^-$'

labels=[label1]
colors=['C4']
data=[
      D0_Truth[(abs(D0_Truth['e_mcPDG'])==11) & (D0_Truth['Ds_extraInfo_BkgBDT']>=0.531) & (D0_Truth['Ds_gammaveto_M_Correction']>=-1) & (D0_Truth['Ds_gammaveto_em_electronID']>=-1)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=2, range=Range)
# plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# plt.axvline(0.16,ls='--',color='gray')
# plt.axvline(0.28,ls='--',color='gray')
# plt.axvspan(0.28,Range[1],color='gray',alpha=0.2)

# Title
#--------
plt.title(r'$D^{*+} \rightarrow [D^{0} \rightarrow K^{-} \pi^{+}] \pi^{+}$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m_{e}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 6)
plt.legend()
plt.show()

Fake $D^{0}$

In [ ]:
dM = -1
FD = -1
BD = -1

FakeD0_Truth = DataFrames["All"][(abs(DataFrames["All"]['D0_isSignal']).isna()) | ((abs(DataFrames["All"]['D0_isSignal'])==0))]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Settings ===
Stacked = False
Density = False
Bins = 50
var = 'Ds_diff_D0pi'
Range = [0.1, 0.55]
BS = -1
dM = 0.06
eID = 0.1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# === Background Histogram (FakeD0_Truth) ===
labels1 = [r'$Other$']
colors1 = ['C5']
data1 = [
    FakeD0_Truth[
        (FakeD0_Truth['Ds_extraInfo_BkgBDT'] >= BS) &
        (FakeD0_Truth['Ds_gammaveto_M_Correction'] <= dM) &
        (FakeD0_Truth['Ds_gammaveto_em_electronID'] >= eID)
    ][var]
]
scales1 = [3.0]  # <-- You can change this to whatever factor you want

plt.hist(
    data1,
    color=colors1,
    label=labels1,
    density=Density,
    stacked=Stacked,
    bins=Bins,
    alpha=1,
    histtype='step',
    linewidth=2,
    range=Range,
    weights=[np.ones_like(d) * s for d, s in zip(data1, scales1)]
)

# === Signal Histograms (from signal MC) ===
labels2 = [r'$isSignal(D_s^{+})=1$ (signal MC)', r'$isSignal(D_s^{+})=0$ (signal MC)']
colors2 = ["#4C6EB1", 'C3']
data2 = [
    DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal'] == 1)][var],
    DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal'] == 0)][var]
]
scales2 = [1.0, 8.0]  # <-- independent scaling for signal MC categories

plt.hist(
    data2[::-1],
    color=colors2[::-1],
    label=labels2[::-1],
    alpha=1,
    range=Range,
    linewidth=2,
    stacked=Stacked,
    density=Density,
    bins=Bins,
    histtype='step',
    weights=[np.ones_like(d) * s for d, s in zip(data2[::-1], scales2[::-1])]
)

plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
plt.axvline(0.16,ls='--',color='gray')
plt.axvline(0.28,ls='--',color='gray')
plt.axvspan(0.28,Range[1],color='gray',alpha=0.2)

# === Titles and labels ===
# plt.title(r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$', loc="left")
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width=perBin))
plt.xlabel(r'$\Delta m_{\pi}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Settings ===
Stacked = False
Density = True
Bins = 50
var = 'e_p'  # now plotting electron momentum
cut_var = 'Ds_diff_D0pi'
cut_min, cut_max = 0.16, 0.28  # non-shaded region
BS = -1
dM = 0.06
eID = 0.1
Samples = "All"
Range = [0.0, 0.6]  # Adjusted for e_p
perBin = ((Range[1] - Range[0]) / Bins) * 1000
print("Width Per Bin: {:.2f} MeV".format(perBin))

# === Background Histogram (FakeD0_Truth) ===
labels1 = [r'$Other$']
colors1 = ['C5']
data1 = [
    FakeD0_Truth[
        (FakeD0_Truth[cut_var] > cut_min) &
        (FakeD0_Truth[cut_var] < cut_max) &
        (FakeD0_Truth['Ds_extraInfo_BkgBDT'] >= BS) &
        (FakeD0_Truth['Ds_gammaveto_M_Correction'] <= dM) &
        (FakeD0_Truth['Ds_gammaveto_em_electronID'] >= eID)
    ][var]
]
scales1 = [5.0]

plt.hist(
    data1,
    color=colors1,
    label=labels1,
    density=Density,
    stacked=Stacked,
    bins=Bins,
    alpha=1,
    histtype='step',
    linewidth=2,
    range=Range,
    weights=[np.ones_like(d) * s for d, s in zip(data1, scales1)]
)

# === Signal Histograms (from signal MC) ===
labels2 = [r'$isSignal(D_s^{+})=1$ (signal MC)', r'$isSignal(D_s^{+})=0$ (signal MC)']
colors2 = ["#4C6EB1", 'C3']
data2 = [
    DataFrames["Signal"][
        (DataFrames["Signal"][cut_var] > cut_min) &
        (DataFrames["Signal"][cut_var] < cut_max) &
        (DataFrames["Signal"]['Ds_isSignal'] == 1)
    ][var],
    
    DataFrames["Signal"][
        (DataFrames["Signal"][cut_var] > cut_min) &
        (DataFrames["Signal"][cut_var] < cut_max) &
        (DataFrames["Signal"]['Ds_isSignal'] == 0)
    ][var]
]
scales2 = [1.0, 8.0]

plt.hist(
    data2[::-1],
    color=colors2[::-1],
    label=labels2[::-1],
    alpha=1,
    range=Range,
    linewidth=2,
    stacked=Stacked,
    density=Density,
    bins=Bins,
    histtype='step',
    weights=[np.ones_like(d) * s for d, s in zip(data2[::-1], scales2[::-1])]
)

# === Labels ===
plt.title(r'$0.16 \leq \Delta m(D_s^{+} - D^0) \leq 0.28$', loc="left")
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c)$'.format(width=perBin))
plt.xlabel(r'$p_{lab}(e^{+})$ [GeV/$c$]')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Settings ===
Stacked = False
Density = False
Bins = 50
var = 'Ds_daughterAngle_0_1'  # angle between e+ and D0
Range = [0, np.pi]
BS = -1
dM = 0.06
eID = 0.1
Samples = "All"
peak_min, peak_max = 0.16, 0.28
perBin = ((Range[1] - Range[0]) / Bins)
print("Width Per Bin: {width:.2f} radians".format(width=perBin))

# === Background Histogram (FakeD0_Truth) in Δm peak region ===
labels1 = [r'$Other$']
colors1 = ['C5']
data1 = [
    FakeD0_Truth[
        (FakeD0_Truth['Ds_diff_D0pi'] > peak_min) &
        (FakeD0_Truth['Ds_diff_D0pi'] < peak_max) &
        (FakeD0_Truth['Ds_extraInfo_BkgBDT'] >= BS) &
        (FakeD0_Truth['Ds_gammaveto_M_Correction'] <= dM) &
        (FakeD0_Truth['Ds_gammaveto_em_electronID'] >= eID)
    ][var]
]
scales1 = [5.0]

plt.hist(
    data1,
    color=colors1,
    label=labels1,
    density=Density,
    stacked=Stacked,
    bins=Bins,
    alpha=1,
    histtype='step',
    linewidth=2,
    range=Range,
    weights=[np.ones_like(d) * s for d, s in zip(data1, scales1)]
)

# === Signal Histograms (from signal MC) over full Δm ===
labels2 = [r'$isSignal(D_s^{+})=1$ (signal MC)', r'$isSignal(D_s^{+})=0$ (signal MC)']
colors2 = ["#4C6EB1", 'C3']
data2 = [
    DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal'] == 1)][var],
    DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal'] == 0)][var]
]
scales2 = [1.0, 8.0]

plt.hist(
    data2[::-1],
    color=colors2[::-1],
    label=labels2[::-1],
    alpha=1,
    range=Range,
    linewidth=2,
    stacked=Stacked,
    density=Density,
    bins=Bins,
    histtype='step',
    weights=[np.ones_like(d) * s for d, s in zip(data2[::-1], scales2[::-1])]
)

# === Titles and labels ===
plt.title(r'$0.16 \leq \Delta m(D_s^{+} - D^0) \leq 0.28$', loc="left")
plt.ylabel(r'$Entries/(\; {width:.2f}\;rad)$'.format(width=perBin))
plt.xlabel(r'$\theta(e^+, D^0)$ [rad]')
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
# var = 'Ds_diff_D0pi'
var = 'e_pt'
Range = [0.0, 0.55]
BS = -1
dM = 0.06
eID = 0.1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Other$'

labels=[label1]
colors=['C5']
data=[
      DataFrames["All"][(DataFrames["All"]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames["All"]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames["All"]['Ds_gammaveto_em_electronID']>=eID)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# plt.axvline(0.16,ls='--',color='gray')

# Title
#--------
plt.title(r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c)$'.format(width = perBin))
# plt.xlabel(r'$\Delta m_{e}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
plt.xlabel(r'$p_{t}(e)\;[GeV/c]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 6000)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = True
# Bins = 50
var = 'Ds_diff_D0pi'
# var = 'Ds_massDifference_0'
# var = 'e_pt'
Range = [0.1, 0.55]
dM = 0.06
FD = -1
BS = -1
eID = 0.1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$BDT \geq 0.531$'
label2= r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$'

labels=[label1,label2]
colors=["#4C6EB1",'C3']
data=[
      FakeD0_Truth[(FakeD0_Truth['Ds_mcPDG']!=421) & (FakeD0_Truth['Ds_extraInfo_BkgBDT']>=0.531) & (FakeD0_Truth['Ds_gammaveto_M_Correction']>=-1) & (FakeD0_Truth['Ds_gammaveto_em_electronID']>=eID)][var],
      FakeD0_Truth[(FakeD0_Truth['Ds_mcPDG']!=421) & (FakeD0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (FakeD0_Truth['Ds_gammaveto_M_Correction']<=dM) & (FakeD0_Truth['Ds_gammaveto_em_electronID']>=eID)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=2, range=Range)
plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
plt.axvline(0.16,ls='--',color='gray')
plt.axvline(0.26,ls='--',color='gray')
plt.axvspan(0.26,Range[1],color='gray',alpha=0.2)

# Title
#--------
plt.title(r'$No \; D^0$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m_{\pi}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 4000)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
# Bins = 50
# var = 'Ds_diff_D0pi'
var = 'Ds_massDifference_0'
# var = 'e_pt'
Range = [0.0, 0.25]
dM = 0.06
FD = -1
BS = -1
eID = 0.1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$BDT \geq 0.531$'
label2= r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$'

labels=[label2]
colors=['C5']
data=[
      FakeD0_Truth[(FakeD0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (FakeD0_Truth['Ds_gammaveto_M_Correction']<=dM) & (FakeD0_Truth['Ds_gammaveto_em_electronID']>=0.1)][var],
      ]

plt.hist(
    data,
    color=colors,
    label=r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$',
    density=Density,
    stacked=Stacked,
    bins=Bins,
    alpha=1,
    histtype='step',
    linewidth=2,
    range=Range
)
# plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# plt.axvline(0.16,ls='--',color='gray')
# plt.axvline(0.26,ls='--',color='gray')
# plt.axvspan(0.26,Range[1],color='gray',alpha=0.2)

# Title
#--------
plt.title(r'$Other$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m_{e}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 4000)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
# Bins = 50
var = 'Ds_diff_D0pi'
# var = 'Ds_massDifference_0'
Range = [0.1, 0.55]
dM = 0.06
FD = -1
BS = -1
eID = 0.1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$BDT \geq 0.531, \; Real \; Electrons$'
label2= r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$'

labels=[label1,label2]
colors=['C5','C3']
data=[
      FakeD0_Truth[(abs(FakeD0_Truth['e_mcPDG'])==11) & (FakeD0_Truth['Ds_extraInfo_BkgBDT']>=0.531) & (FakeD0_Truth['Ds_gammaveto_M_Correction']>=-1) & (FakeD0_Truth['Ds_gammaveto_em_electronID']>=-1)][var],
      FakeD0_Truth[(FakeD0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (FakeD0_Truth['Ds_gammaveto_M_Correction']<=dM) & (FakeD0_Truth['Ds_gammaveto_em_electronID']>=eID)][var],
      ]

scaling_factors = [1.0, 0.045]  # Scale first histogram normally, second one is halved
weights = [
    scaling_factors[0] * np.ones_like(data[0]),
    scaling_factors[1] * np.ones_like(data[1])
]
plt.hist(
    data,
    weights=weights,
    color=colors,
    label=labels,
    density=Density,
    stacked=Stacked,
    bins=Bins,
    alpha=1,
    histtype='step',
    linewidth=2,
    range=Range
)
plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
plt.axvline(0.16,ls='--',color='gray')
plt.axvline(0.28,ls='--',color='gray')
plt.axvspan(0.28,Range[1],color='gray',alpha=0.2)

# Title
#--------
plt.title(r'Other', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m_{\pi}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 4)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
# Bins = 50
var = 'Ds_diff_D0pi'
# var = 'Ds_massDifference_0'
Range = [0.1, 0.55]
dM = 0.06
FD = -1
BS = -1
eID = 0.1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1 = r'$BDT \geq 0.531$, Truth-matched $e^-$'

labels=[label1]
colors=['C5']
data=[
      FakeD0_Truth[(abs(FakeD0_Truth['e_mcPDG'])==11) & (FakeD0_Truth['Ds_extraInfo_BkgBDT']>=0.531) & (FakeD0_Truth['Ds_gammaveto_M_Correction']>=-1) & (FakeD0_Truth['Ds_gammaveto_em_electronID']>=-1)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=2, range=Range)
# plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# plt.axvline(0.16,ls='--',color='gray')
# plt.axvline(0.28,ls='--',color='gray')
# plt.axvspan(0.28,Range[1],color='gray',alpha=0.2)

# Title
#--------
plt.title(r'Other', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m_{\pi}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 6)
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np  # needed for np.nan

# === USER OPTIONS ===
Density = False       # Normalize histograms
Bins = 50
Range = [0.0, 0.25]   # GeV
eID = 0.1             # Electron ID threshold
dM = 0.06             # m(ee) cut
# variable = 'Ds_diff_D0pi'
variable = 'Ds_massDifference_0'
perBin = ((Range[1] - Range[0]) / Bins) * 1000
print(f"Width Per Bin: {perBin:.2f} MeV")

# === COLORS & LABELS ===
sub_labels = [
    r'$Fake \; D^0$',   # Combined 23 + 300553
    r'$Real \; D^0$',
    r'NaN'
]
sub_colors = ["#007C91", "#4C6EB1", "#2E2E2E"]  # dark cyan, blue, gray

# === MAIN CATEGORY CONFIGURATION ===
main_categories = [
    ("Other", FakeD0_Truth),
]

# === LOOP OVER MAIN CATEGORIES ===
for title_label, df in main_categories:
    fig, ax = plt.subplots(figsize=(6.5, 5))

    # Prepare the 3 subcategory histograms
    hists = []

    # 1. Combined (|23| or |300553|)
    base_mask = (
        (df['Ds_gammaveto_em_electronID'] >= eID) &
        (df['Ds_gammaveto_M_Correction'] <= dM)
    )
    comb_mask = base_mask & (
        (df['D0_mcPDG'].abs() == 23) | (df['D0_mcPDG'].abs() == 300553)
    )
    hists.append(df[comb_mask][variable])

    # 2. 421
    mask_421 = base_mask & (df['D0_mcPDG'].abs() == 421)
    hists.append(df[mask_421][variable])

    # 3. NaN
    mask_nan = base_mask & (df['D0_mcPDG'].isna())
    hists.append(df[mask_nan][variable])

    # Plot
    ax.hist(hists,
            bins=Bins,
            range=Range,
            density=Density,
            stacked=False,
            histtype='step',
            linewidth=1.5,
            color=sub_colors,
            label=sub_labels)

    # Axis and title formatting
    ax.set_xlabel(r'$\Delta m_{e}(D_s^{+} - D^0)\;[\mathrm{GeV}/c^2]$')
    ax.set_ylabel(r'$Entries / ({:.2f}\; \mathrm{{MeV}}/c^2)$'.format(perBin))
    ax.set_title(title_label, loc='left', fontsize=11)
    ax.set_title(r'$\int\mathcal{L}dt \approx 1444$ fb$^{-1}$', loc='right')
    ax.legend(loc='upper right')
    # plt.tight_layout()
    plt.show()


In [ ]:
Stacked = False
Density = True
Bins = 50
# var = 'Ds_diff_D0pi'
# var = 'Ds_massDifference_0'
var = 'e_pt'
Range = [0.0, 0.6]
dM = 0.06
FD = -1
BS = -1
eID = 0.1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$BDT \geq 0.531$'
label2= r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$'

labels=[label1,label2]
colors=["#A63F00", "#6A3D9A"]
data=[
      FakeD0_Truth[(abs(FakeD0_Truth['e_mcPDG'])==11) & (FakeD0_Truth['Ds_extraInfo_BkgBDT']>=0.531) & (FakeD0_Truth['Ds_gammaveto_M_Correction']>=dM) & (FakeD0_Truth['Ds_gammaveto_em_electronID']>=eID)][var],
      FakeD0_Truth[(abs(FakeD0_Truth['e_mcPDG'])==11) & (FakeD0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (FakeD0_Truth['Ds_gammaveto_M_Correction']<=dM) & (FakeD0_Truth['Ds_gammaveto_em_electronID']>=eID)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=2, range=Range)
# plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# plt.axvline(0.16,ls='--',color='gray')

# Title
#--------
plt.title(r'Other', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m_{e}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 30000)
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Extract your dataframe
df = DataFrames["All"]

# Apply the dielectron mass cut (Photon Conversion Region)
# You may need to adjust the name of the dielectron mass variable
# I will assume it's something like 'dielectron_mass'
# If your dielectron mass variable is named differently, change 'dielectron_mass' to your column name
dielectron_mass_var = 'Ds_gammaveto_M_Correction'

# Select events inside the dielectron invariant mass peak
mask_dielectron_peak = ((abs(df['D0_isSignal']).isna()) | ((abs(df['D0_isSignal'])==0))) & (df[dielectron_mass_var] > 0.06) & (df['Ds_gammaveto_em_electronID']>=0.1)
df_selected = df[mask_dielectron_peak].copy()

# Compute the Δm shift (if not already computed)
df_selected["delta_m_shift"] = df_selected["Ds_diff_D0pi"] - df_selected["Ds_massDifference_0"]

# Set ranges and binning
p_min, p_max = 0.0, 0.7  # GeV/c
shift_min, shift_max = 0.0, 0.3  # GeV/c^2

n_bins_p = 70  # momentum bins (10 MeV wide)
n_bins_shift = 70  # shift axis bins

# Create the 2D histogram
plt.figure(figsize=(8, 6))
hist = plt.hist2d(
    df_selected["e_p"], df_selected["delta_m_shift"],
    bins=[n_bins_p, n_bins_shift],
    range=[[p_min, p_max], [shift_min, shift_max]],
    cmap="viridis"
)

# Add colorbar and labels
plt.colorbar(label="Entries")
plt.xlabel("Electron Candidate Momentum $p_{lab}(e^{+})$ [GeV/$c$]", fontsize=13)
plt.ylabel(r"Shift in $\Delta m$ ($\Delta m_\pi - \Delta m_e$) [GeV/$c^2$]", fontsize=13)
plt.title(r"Photon Conversion Region: $\Delta m$ Shift vs Electron Momentum", fontsize=14)
plt.tight_layout()
plt.show()


All Background

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# === Filter to true electrons only ===
df = DataFrames["uds"][
    (DataFrames["uds"]["Ds_gammaveto_M_Correction"] <= 0.06)
]

# === Build 3-momentum arrays ===
p_D0 = df[["D0_px", "D0_py", "D0_pz"]].to_numpy()
p_e  = df[["e_px", "e_py", "e_pz"]].to_numpy()
m_D0_array = df["D0_M"].to_numpy()

# === Energy computation ===
def compute_energy(p, m):
    return np.sqrt(np.sum(p**2, axis=1) + m**2)

# === Δm computation for given mass hypothesis ===
def compute_delta_m(p_D0, p_e, m_D0_array, m_e_hyp):
    E_D0 = compute_energy(p_D0, m_D0_array)
    E_e  = compute_energy(p_e, m_e_hyp)
    p_Ds = p_D0 + p_e
    E_Ds = E_D0 + E_e
    p_Ds2 = np.sum(p_Ds**2, axis=1)
    m_Ds = np.sqrt(E_Ds**2 - p_Ds2)
    return m_Ds - m_D0_array

# === Compute Δm shift for pion vs electron mass
delta_m_pi = compute_delta_m(p_D0, p_e, m_D0_array, 0.1396)
delta_m_e  = compute_delta_m(p_D0, p_e, m_D0_array, 0.000511)
delta_m_shift = delta_m_pi - delta_m_e

# === Electron magnitude
e_p_magnitude = np.linalg.norm(p_e, axis=1)

# === cosθ between e⁺ and D⁰
dot_products = np.sum(p_D0 * p_e, axis=1)
norms_product = np.linalg.norm(p_D0, axis=1) * np.linalg.norm(p_e, axis=1)
cos_theta = dot_products / norms_product

# === Manual plotting ranges
p_min, p_max = 0.0, 0.7
cos_min, cos_max = -1.0, 1.0
shift_min, shift_max = 0.0, 0.3
n_bins = 70

# === Plot 1: Δm shift vs p(e⁺)
plt.figure(figsize=(8, 6))
plt.hist2d(
    e_p_magnitude,
    delta_m_shift,
    bins=[n_bins, n_bins],
    range=[[p_min, p_max], [shift_min, shift_max]],
    cmap="viridis"
)
plt.colorbar(label="Entries")
plt.xlabel("Electron Candidate Momentum $p_{lab}(e^{+})$ [GeV/$c$]", fontsize=12)
plt.ylabel(r"Shift in $\Delta m$ ($\Delta m_{\pi} - \Delta m_{e}$) [GeV/$c^2$]", fontsize=12)
plt.title(r"Only True Electrons: $\Delta m$ Shift vs $p(e^+)$ (Pion Hypothesis)", fontsize=14)
plt.tight_layout()
plt.show()

# === Plot 2: Δm shift vs cosθ(e⁺, D⁰)
plt.figure(figsize=(8, 6))
plt.hist2d(
    cos_theta,
    delta_m_shift,
    bins=[n_bins, n_bins],
    range=[[cos_min, cos_max], [shift_min, shift_max]],
    cmap="plasma"
)
plt.colorbar(label="Entries")
plt.xlabel(r"cos$\theta(e^{+}, D^{0})$", fontsize=12)
plt.ylabel(r"Shift in $\Delta m$ ($\Delta m_{\pi} - \Delta m_{e}$) [GeV/$c^2$]", fontsize=12)
plt.title(r"Only True Electrons: $\Delta m$ Shift vs cosθ(e⁺, D⁰) (Pion Hypothesis)", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# === USER OPTIONS ===
Density = True      # Set to True to normalize histograms
Bins = 50
Range = [0.0, 0.25]   # GeV
dM = 0.06             # Cut on m(ee)
eID = 0.0             # Electron ID threshold
Samples = "All"
variable = "Ds_massDifference_0"
perBin = ((Range[1] - Range[0]) / Bins) * 1000
print(f"Width Per Bin: {perBin:.2f} MeV")

# === LABELS & COLORS ===
labels = [
    r'$D^{*0} \rightarrow D^{0} [\pi^0 \rightarrow \gamma [\gamma \rightarrow e^{+} e^{-}]]$',
    r'$D^{*0} \rightarrow D^{0} [\pi^0 \rightarrow \gamma e^{+} e^{-}]$',
    r'$D^{*0} \rightarrow D^{0} [\gamma \rightarrow e^{+} e^{-}]$'
]
# Scientific color pair for the two regions
region_colors = ["#007C91", "#4C6EB1"]  # dark cyan, steel blue

# === CATEGORY DEFINITIONS ===
category_masks = [
    (abs(DataFrames[Samples]['Ds_mcPDG']) == 423) &
    (abs(DataFrames[Samples]['Ds_genNStepsToDaughter_1']) == 3) &
    (abs(DataFrames[Samples]['e_genMotherPDG']) == 22),

    (abs(DataFrames[Samples]['Ds_mcPDG']) == 423) &
    (abs(DataFrames[Samples]['Ds_genNStepsToDaughter_1']) == 2) &
    (abs(DataFrames[Samples]['e_genMotherPDG']) == 111),

    (abs(DataFrames[Samples]['Ds_mcPDG']) == 423) &
    (abs(DataFrames[Samples]['Ds_genNStepsToDaughter_1']) == 2) &
    (abs(DataFrames[Samples]['e_genMotherPDG']) == 22),
]

# === LOOP TO CREATE 3 SEPARATE PLOTS ===
for i, (mask, label) in enumerate(zip(category_masks, labels)):
    base = DataFrames[Samples][mask & (DataFrames[Samples]['Ds_gammaveto_em_electronID'] >= eID)]

    low_data = base[base['Ds_gammaveto_M_Correction'] <= dM][variable]
    high_data = base[base['Ds_extraInfo_BkgBDT'] > 0.531][variable]

    # plt.figure(figsize=(6.5, 5))
    plt.hist([low_data, high_data],
             bins=Bins,
             range=Range,
             density=Density,
             histtype='step',
             linewidth=2,
             color=region_colors,
             label=[
                 r'$m(e^{+}_{sig} e^{-}_{ROE}) \leq 0.06\; \mathrm{GeV}/c^2$',
                 r'$BDT \geq 0.531$'
             ])

    # Axis and label formatting
    plt.xlabel(r'$\Delta m_{e}(D_s^{+} - D^{0})\;[\mathrm{GeV}/c^2]$')
    plt.ylabel(r'$Entries / ({:.2f}\; \mathrm{{MeV}}/c^2)$'.format(perBin))
    plt.title(label, loc='left', fontsize=12)
    plt.title(r'$\int\mathcal{L}dt \approx 1444$ fb$^{-1}$', loc='right')
    plt.legend(fontsize=12)
    plt.tight_layout()
    plt.show()


In [ ]:
Stacked = False
Density = False
Bins = 50
i = 'Ds_massDifference_0'
# i = 'Ds_diff_D0pi'
Range = [0.0,0.25]
dM = 0.06
BD = -1
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$D^{*0} \rightarrow D^{0} [\pi^0 \rightarrow \gamma [\gamma \rightarrow e^{+} e^{-}]]$'
label2= r'$D^{*0} \rightarrow D^{0} [\pi^0 \rightarrow \gamma  e^{+} e^{-}]$'
label3= r'$D^{*0} \rightarrow D^{0} [\gamma \rightarrow  e^{+} e^{-}]$'

labels=[label1,label2,label3]
data=[
      DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])==423) & (abs(DataFrames["All"]['Ds_genNStepsToDaughter_1'])==3) & (abs(DataFrames["All"]['e_genMotherPDG'])==22)  & (DataFrames["All"]['Ds_gammaveto_M_Correction']<=dM) & ((DataFrames["All"]["Ds_extraInfo_BkgBDT"])>=BD)][i],
      DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])==423) & (abs(DataFrames["All"]['Ds_genNStepsToDaughter_1'])==2) & (abs(DataFrames["All"]['e_genMotherPDG'])==111) & (DataFrames["All"]['Ds_gammaveto_M_Correction']<=dM) & ((DataFrames["All"]["Ds_extraInfo_BkgBDT"])>=BD)][i],
      DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])==423) & (abs(DataFrames["All"]['Ds_genNStepsToDaughter_1'])==2) & (abs(DataFrames["All"]['e_genMotherPDG'])==22)  & (DataFrames["All"]['Ds_gammaveto_M_Correction']<=dM) & ((DataFrames["All"]["Ds_extraInfo_BkgBDT"])>=BD)][i],
      ]

# factor = 1
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']<=dM) & ((DataFrames["Mode_2"]["Ds_extraInfo_BkgBDT"])>=BD)][i], color=["#D62728"], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']<=dM) & ((DataFrames["Mode_2"]["Ds_extraInfo_BkgBDT"])>=BD)][i]), linewidth=1.5)
plt.hist(data, color= ["#007C91", "#4C6EB1", "#2E2E2E"], label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.15,ls='--',color='gray')

# Title
#--------
plt.title(r'$m(e_{sig}^{+} e_{ROE}^{-}) \geq 0.06 GeV/c^{2}$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# === Analysis parameters ===
Samples = "uds"
var_x = "Ds_daughterAngle_0_1"   # in radians
var_y = "Ds_diff_D0pi"           # ∆m with electron mass
Bins_x = 70
Bins_y = 70
Range_x = [-1, 1]    # cos(theta)
Range_y = [0.1, 0.55]  # ∆m range in GeV

dM = 0.06
BS = -1
eID = -1

# === Apply selection
base_df = DataFrames[Samples][
    (DataFrames[Samples]['Ds_extraInfo_BkgBDT'] >= BS) &
    (DataFrames[Samples]['Ds_gammaveto_M_Correction'] <= dM) &
    (DataFrames[Samples]['Ds_gammaveto_em_electronID'] >= eID)
]

# === Compute cos(theta) from angle in radians
cos_theta = np.cos(base_df[var_x])
delta_m = base_df[var_y]

# === 2D Histogram
# plt.figure(figsize=(8, 6))
plt.hist2d(
    cos_theta,
    delta_m,
    bins=[Bins_x, Bins_y],
    range=[Range_x, Range_y],
    cmap="coolwarm"
)

plt.colorbar(label="Entries")
plt.xlabel(r'$\cos(\theta)(D^0 e^+)$')
plt.ylabel(r'$\Delta m_{\pi}(D_s^+ - D^0)\;[\mathrm{GeV}/c^2]$')
plt.title(r'2D Distribution: $\cos(\theta)$ vs $\Delta m_{\pi}$ (uds)')
# plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# === Selection parameters
Samples = "uds"
dM = 0.06
BS = -1
eID = -1

# === Apply selection
base_df = DataFrames[Samples][
    (DataFrames[Samples]['Ds_extraInfo_BkgBDT'] >= BS) &
    (DataFrames[Samples]['Ds_gammaveto_M_Correction'] <= dM) &
    (DataFrames[Samples]['Ds_gammaveto_em_electronID'] >= eID)
]

# === Build variables
delta_m_e  = base_df["Ds_massDifference_0"]
delta_m_pi = base_df["Ds_diff_D0pi"]
angle_rad  = base_df["Ds_daughterAngle_0_1"]
p_e        = np.linalg.norm(base_df[["e_px", "e_py", "e_pz"]].to_numpy(), axis=1)

# === Assemble into DataFrame for correlation
cor_df = pd.DataFrame({
    r"$\Delta m_{e}$ [GeV/$c^2$]": delta_m_e,
    r"$\Delta m_{\pi}$ [GeV/$c^2$]": delta_m_pi,
    r"$\theta(D^0, e^+)$ [rad]": angle_rad,
    r"$p(e^+)$ [GeV/$c$]": p_e
})

# === Compute correlation matrix
corr_matrix = cor_df.corr(method='pearson')

# === Plot heatmap
plt.figure(figsize=(7, 6))
im = plt.imshow(corr_matrix, cmap="coolwarm", vmin=-1, vmax=1)

# Axis ticks and labels
plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=45, ha='right')
plt.yticks(range(len(corr_matrix.index)), corr_matrix.index)

# Annotate with correlation values
for i in range(len(corr_matrix.columns)):
    for j in range(len(corr_matrix.index)):
        value = corr_matrix.iloc[j, i]
        plt.text(i, j, f"{value:.2f}", ha='center', va='center', color='black')

plt.colorbar(im, label="Pearson Correlation")
plt.title("Correlation Matrix (uds events)")
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# === Selection parameters
Samples = "Signal"
dM = 0.06
BS = -1
eID = -1

# === Apply selection
base_df = DataFrames[Samples]

# === Build all variables
variables = {
    r"$\Delta m_{e}$ [GeV/$c^2$]": base_df["Ds_massDifference_0"],
    r"$\Delta m_{\pi}$ [GeV/$c^2$]": base_df["Ds_diff_D0pi"],
    r"$\Delta m_{K}$ [GeV/$c^2$]": base_df["Ds_diff_D0K"],
    r"$\Delta m_{e}^{\mathrm{noVF}}$ [GeV/$c^2$]": base_df["Ds_diff_D0e_noVF"],
    r"$\Delta m_{\pi}^{\mathrm{noVF}}$ [GeV/$c^2$]": base_df["Ds_diff_D0pi_noVF"],
    r"$\Delta m_{K}^{\mathrm{noVF}}$ [GeV/$c^2$]": base_df["Ds_diff_D0K_noVF"],
    r"$\cos\theta(D^0,e^+)^{\mathrm{noVF}}$": base_df["Ds_cos_theta_D0e_noVF"],
    r"$\cos\theta_{\mathrm{open}}(D^0,e^+)$": base_df["Ds_cos_daughterAngle_0_1"],
#     r"$\theta_{\mathrm{open}}(D^0,e^+)$ [rad]": base_df["Ds_daughterAngle_0_1"],
#     r"$p(e^+)$ [GeV/$c$]": np.linalg.norm(base_df[["e_px", "e_py", "e_pz"]].to_numpy(), axis=1),
}

# === Assemble DataFrame for correlation
cor_df = pd.DataFrame(variables)

# === Compute correlation matrix
corr_matrix = cor_df.corr(method='pearson')

# === Plot heatmap
plt.figure(figsize=(10, 8))
im = plt.imshow(corr_matrix, cmap="coolwarm", vmin=-1, vmax=1)

# Axis ticks and labels
plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=45, ha='right')
plt.yticks(range(len(corr_matrix.index)), corr_matrix.index)

# Annotate with correlation values
for i in range(len(corr_matrix.columns)):
    for j in range(len(corr_matrix.index)):
        value = corr_matrix.iloc[j, i]
        plt.text(i, j, f"{value:.2f}", ha='center', va='center', color='black')

plt.colorbar(im, label="Pearson Correlation")
plt.title("Correlation Matrix (uds events)")
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# === Selection parameters
Samples = "uds"
dM = 0.06
BS = -1
eID = -1

# === Apply selection
base_df = DataFrames[Samples][
    (DataFrames[Samples]['Ds_extraInfo_BkgBDT'] >= BS) &
    (DataFrames[Samples]['Ds_gammaveto_M_Correction'] <= dM) &
    (DataFrames[Samples]['Ds_gammaveto_em_electronID'] >= eID)
]

# === Build all variables
variables = {
    r"$\Delta m_{e}$ [GeV/$c^2$]": base_df["Ds_massDifference_0"],
    r"$\Delta m_{\pi}$ [GeV/$c^2$]": base_df["Ds_diff_D0pi"],
    r"$\Delta m_{K}$ [GeV/$c^2$]": base_df["Ds_diff_D0K"],
    r"$\Delta m_{e}^{\mathrm{noVF}}$ [GeV/$c^2$]": base_df["Ds_diff_D0e_noVF"],
    r"$\Delta m_{\pi}^{\mathrm{noVF}}$ [GeV/$c^2$]": base_df["Ds_diff_D0pi_noVF"],
    r"$\Delta m_{K}^{\mathrm{noVF}}$ [GeV/$c^2$]": base_df["Ds_diff_D0K_noVF"],
    r"$\cos\theta(D^0,e^+)^{\mathrm{noVF}}$": base_df["Ds_cos_theta_D0e_noVF"],
    r"$\cos\theta_{\mathrm{open}}(D^0,e^+)$": base_df["Ds_cos_daughterAngle_0_1"],
#     r"$\theta_{\mathrm{open}}(D^0,e^+)$ [rad]": base_df["Ds_daughterAngle_0_1"],
#     r"$p(e^+)$ [GeV/$c$]": np.linalg.norm(base_df[["e_px", "e_py", "e_pz"]].to_numpy(), axis=1),
}

# === Assemble DataFrame for correlation
cor_df = pd.DataFrame(variables)

# === Compute correlation matrix
corr_matrix = cor_df.corr(method='pearson')

# === Plot heatmap
plt.figure(figsize=(10, 8))
im = plt.imshow(corr_matrix, cmap="coolwarm", vmin=-1, vmax=1)

# Axis ticks and labels
plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=45, ha='right')
plt.yticks(range(len(corr_matrix.index)), corr_matrix.index)

# Annotate with correlation values
for i in range(len(corr_matrix.columns)):
    for j in range(len(corr_matrix.index)):
        value = corr_matrix.iloc[j, i]
        plt.text(i, j, f"{value:.2f}", ha='center', va='center', color='black')

plt.colorbar(im, label="Pearson Correlation")
plt.title("Correlation Matrix (uds events)")
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# === Selection parameters
Samples = "uds"
dM = 0.06
BS = -1
eID = -1

# === Apply selection
base_df = DataFrames[Samples][
    (DataFrames[Samples]['Ds_extraInfo_BkgBDT'] >= BS) &
    (DataFrames[Samples]['Ds_gammaveto_M_Correction'] <= dM) &
    (DataFrames[Samples]['Ds_gammaveto_em_electronID'] >= eID)
]

# === Mass hypothesis Δm variables
delta_m_e     = base_df["Ds_massDifference_0"]
delta_m_pi    = base_df["Ds_diff_D0pi"]
delta_m_K     = base_df["Ds_diff_D0K"]

delta_m_e_noVF  = base_df["Ds_diff_D0e_noVF"]
delta_m_pi_noVF = base_df["Ds_diff_D0pi_noVF"]
delta_m_K_noVF  = base_df["Ds_diff_D0K_noVF"]

# === Angular variables
cos_theta_noVF = base_df["Ds_cos_theta_D0e_noVF"]
cos_theta_VF   = base_df["Ds_cos_daughterAngle_0_1"]

# === Plot Δm comparisons
mass_hypotheses = {
    "Electron": (delta_m_e, delta_m_e_noVF, [0.0, 0.25], "Δm_e"),
    "Pion":     (delta_m_pi, delta_m_pi_noVF, [0.1, 0.55], "Δm_π"),
    "Kaon":     (delta_m_K, delta_m_K_noVF, [0.2, 1.25], "Δm_K"),
}

for label, (vf, no_vf, xlim, tag) in mass_hypotheses.items():
    plt.figure(figsize=(7, 5))
    plt.hist(vf,     bins=70, range=xlim, histtype='step', linewidth=2, label=f"{label} (VF)", color='C0')
    plt.hist(no_vf,  bins=70, range=xlim, histtype='step', linewidth=2, label=f"{label} (noVF)", color='C1')
    plt.xlabel(f"{tag} [GeV/$c^2$]")
    plt.ylabel("Entries")
    plt.title(f"{tag} with and without Vertex Fitting")
    plt.legend()
    plt.tight_layout()
    plt.show()

# === Plot angle comparisons
plt.figure(figsize=(7, 5))
plt.hist(cos_theta_VF, bins=70, range=[-1, 1], histtype='step', linewidth=2, label="cosθ (VF)", color='C0')
plt.hist(cos_theta_noVF, bins=70, range=[-1, 1], histtype='step', linewidth=2, label="cosθ (noVF)", color='C1')
plt.xlabel(r"$\cos\theta(D^0, e^+)$")
plt.ylabel("Entries")
plt.title("cosθ with and without Vertex Fitting")
plt.legend()
plt.tight_layout()
plt.show()

# plt.figure(figsize=(7, 5))
# plt.hist(theta_open, bins=70, range=[0, np.pi], histtype='step', linewidth=2, label=r"$\theta_{\text{open}}$", color='C2')
# plt.xlabel(r"$\theta_{\mathrm{open}}(D^0, e^+)$ [rad]")
# plt.ylabel("Entries")
# plt.title(r"$\theta_{\mathrm{open}}$ distribution (VF)")
# plt.legend()
# plt.tight_layout()
# plt.show()


In [ ]:
import numpy as np
import pandas as pd

# === Filter to true electrons only
df = DataFrames["uds"][
    (DataFrames["uds"]["Ds_gammaveto_M_Correction"] <= 0.06)
].copy()

# === invmass function (your version)
def invmass(m1, m, p1, p2, px1, px2, py1, py2, pz1, pz2):
    if m == 'pi':
        m2 = 0.13957039
    elif m == 'e':
        m2 = 0.00051099895
    elif m == 'K':
        m2 = 0.493677
    elif m == 'p':
        m2 = 0.938272
    else:
        raise ValueError("Unsupported mass hypothesis.")
        
    E1 = np.sqrt(m1**2 + p1**2)
    E2 = np.sqrt(m2**2 + p2**2)

    P1 = np.array([px1, py1, pz1])
    P2 = np.array([px2, py2, pz2])

    P = p1**2 + p2**2 + (2 * np.dot(P1, P2))

    return np.sqrt((E1 + E2)**2 - P)

# === Compute manual Δm (electron mass hypothesis)
df["DeltaM_manual"] = df.apply(lambda row: invmass(
    row["D0_M"], 'e', row["D0_p"], row["e_p"],
    row["D0_px"], row["e_px"],
    row["D0_py"], row["e_py"],
    row["D0_pz"], row["e_pz"]
) - row["D0_M"], axis=1)

# === Get basf2 version
df["DeltaM_basf2"] = df["Ds_massDifference_0"]

# === Compute Pearson correlation
correlation = df[["DeltaM_manual", "DeltaM_basf2"]].corr(method="pearson").iloc[0, 1]
print(f"Pearson correlation between Δm_manual and Δm_basf2: {correlation:.4f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Stacked = False
Density = False
Bins = 50
var = "Ds_daughterAngle_0_1"
Range = [-1, 1]
dM = 0.06
FD = -1
BS = -1
eID = -1
Samples = "uds"
perBin = ((Range[1] - Range[0]) / Bins) * 1000
print("Width Per Bin: {width:.2f} MeV".format(width=perBin))

label1 = r'$uds$'
labels = [label1]
colors = ["#007C91"]

# Apply all cuts
base_df = DataFrames[Samples][
    (DataFrames[Samples]['Ds_extraInfo_BkgBDT'] >= BS) &
    (DataFrames[Samples]['Ds_gammaveto_M_Correction'] <= dM) &
    (DataFrames[Samples]['Ds_gammaveto_em_electronID'] >= eID)
]

# Apply cosine transformation to angle
cos_theta = np.cos(base_df[var])
data = [cos_theta]

# Plotting
plt.hist(
    data,
    color=colors,
    label=labels,
    density=Density,
    stacked=Stacked,
    bins=Bins,
    alpha=1,
    histtype='step',
    linewidth=2,
    range=Range
)

# Title
plt.title(r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$', loc="left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc="right")

# Labels
plt.ylabel(r'$Entries$')
plt.xlabel(r'$\cos(\theta)(D^{0} e^+)$')

plt.legend()
# plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# === Cosine cut threshold ===
cos_theta_cut = 2

# === Variables and bin settings ===
Stacked = False
Density = False
Bins = 50
var = 'Ds_diff_D0pi_noVF'
Range = [0.1, 0.55]
dM = 0.05
FD = -1
BS = -1
eID = -1
Samples = "uds"

# === Per-bin width in MeV
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width=perBin))

# === Label and color setup
label1 = r'$uds$'
labels = [label1]
colors = ["#007C91"]

# === Cosine-based angle cut
angle_cos = np.cos(DataFrames[Samples]["Ds_daughterAngle_0_1"])
mask = (
    (angle_cos <= cos_theta_cut) &
    (DataFrames[Samples]["e_pt"] >= 0.1) &
    (DataFrames[Samples]["Ds_extraInfo_BkgBDT"] >= BS) &
    (DataFrames[Samples]["Ds_gammaveto_M_Correction"] <= dM) &
    (DataFrames[Samples]["Ds_gammaveto_em_electronID"] >= eID)
)

# === Pull variable
data = [DataFrames[Samples][mask][var]]

# === Plotting
plt.hist(
    data,
    color=colors,
    label=labels,
    density=Density,
    stacked=Stacked,
    bins=Bins,
    alpha=1,
    histtype='step',
    linewidth=2,
    range=Range
)

# Titles and labels
plt.title(r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$' + '\n' + '$\cos \Theta (D^{0} e^+) \leq 0.75$', loc="left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc="right")
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width=perBin))
plt.xlabel(r'$\Delta m_{\pi}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
plt.legend()
# plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Configs
Stacked = False
Density = False
Bins = 50
Range = [0.1, 0.55]  # GeV
dM = 0.06
FD = -1
BS = -1
eID = -1
Samples = "uds"

# Bin width for labeling
perBin = ((Range[1] - Range[0]) / Bins) * 1000  # MeV
print("Width Per Bin: {width:.2f} MeV".format(width=perBin))

# =============================
# Extract the relevant arrays
# =============================

df = DataFrames[Samples][
    (DataFrames[Samples]['Ds_extraInfo_BkgBDT'] >= BS) &
    (DataFrames[Samples]['Ds_gammaveto_M_Correction'] <= dM) &
    (DataFrames[Samples]['Ds_gammaveto_em_electronID'] >= eID)
]

dm_pi = df['Ds_diff_D0pi'].to_numpy()
dm_e = df['Ds_massDifference_0'].to_numpy()
pe = df['e_p'].to_numpy()  # Adjust this if your momentum variable has a different name

# =============================
# Plot Δm_π - Δm_e
# =============================

plt.figure(figsize=(7, 5))
plt.hist((dm_pi - dm_e), range=[-0.5,0.5], bins=50, histtype='step', linewidth=2, color="#E84A27")
plt.xlabel(r'$\Delta m_{\pi} - \Delta m_{e}$ [MeV/$c^{2}$]')
plt.ylabel('Entries')
plt.title(r'Difference in $\Delta m$ from Pion vs Electron Mass Hypothesis', loc='left')
# plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc='right')
plt.tight_layout()
plt.show()

# =============================
# Plot 2D histogram: Δm_π vs p_e
# =============================

plt.figure(figsize=(8, 6))
plt.hist2d(pe, dm_pi, bins=[60, 60], range=[[0, 0.5], [Range[0], Range[1]]], cmap='viridis')
cbar = plt.colorbar()
cbar.set_label("Entries")
plt.xlabel(r'$p_{e^{+}_{sig}}$ [GeV/$c$]')
plt.ylabel(r'$\Delta m_{\pi}(D_s^+ - D^0)$ [GeV/$c^{2}$]')
plt.title(r'$\Delta m_{\pi}$ vs $p_e$ with pion mass hypothesis')
plt.tight_layout()
plt.show()


In [ ]:
# 2D plot: Δm_e vs p_e
plt.figure(figsize=(8, 6))
plt.hist2d(pe, dm_e, bins=[60, 60], range=[[0, 0.5], [0.0, 0.4]], cmap='viridis')
cbar = plt.colorbar()
cbar.set_label("Entries")
plt.xlabel(r'$p_{e^{+}_{sig}}$ [GeV/$c$]')
plt.ylabel(r'$\Delta m_{e}(D_s^+ - D^0)$ [GeV/$c^{2}$]')
plt.title(r'$\Delta m_{e}$ vs $p_e$ with correct mass hypothesis')
plt.tight_layout()
plt.show()

In [ ]:
print(abs(DataFrames[Samples][(DataFrames[Samples]['Ds_diff_D0pi']<=0.27) & (DataFrames[Samples]['Ds_diff_D0pi']>=0.16) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames[Samples]['Ds_gammaveto_em_electronID']>=eID)]['e_mcPDG']).value_counts(normalize=True,dropna=False).apply(lambda x: f"{x:.6f}"))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# === Settings ===
Stacked = False
Density = True
Bins = 50
Range = [0.0, 0.6]  # Electron momentum range in GeV/c
Samples = "uds"
DeltaM_Low, DeltaM_High = 0.16, 0.27

# === Momentum bin width ===
perBin = ((Range[1] - Range[0]) / Bins) * 1000
print("Width Per Bin: {width:.2f} MeV".format(width=perBin))

# === Extract dataframe and variables ===
df = DataFrames[Samples]
delta_m = df["Ds_diff_D0pi"]
p_e = df["e_p"]

# === Region definitions ===
in_non_gray = (delta_m > DeltaM_Low) & (delta_m < DeltaM_High)
in_gray = ~in_non_gray

# === Data selections ===
p_non_gray = p_e[in_non_gray]
p_gray = p_e[in_gray]

# === Histogram Plot ===
data = [p_non_gray, p_gray]
colors = ["#007C91", "#2E2E2E"]
labels = [
    r"uds ($0.16 < \Delta m_{\pi}(D_s^+ - D^0) < 0.27$ GeV/$c^2$)",
    r"uds (Outside peak region)"
]

plt.hist(
    data,
    bins=Bins,
    range=Range,
    stacked=Stacked,
    density=Density,
    histtype="step",
    linewidth=2,
    color=colors,
    label=labels
)

# === Titles ===
plt.title(r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc="right")

# === Axis Labels ===
plt.xlabel(r"$p(e^+)$ [GeV/$c$]")
plt.ylabel(r"$Entries/(\;{:.2f}\;MeV/c$)".format(perBin))
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# === Settings ===
Stacked = False
Density = True
Bins = 50
Range = [0.0, 3.2]  # Angle range in radians
Samples = "uds"
DeltaM_Low, DeltaM_High = 0.16, 0.27

# === Bin width (in radians) ===
perBin = ((Range[1] - Range[0]) / Bins)
print("Width Per Bin: {width:.3f} rad".format(width=perBin))

# === Extract dataframe and variables ===
df = DataFrames[Samples]
delta_m = df["Ds_diff_D0pi"]
angle = df["Ds_daughterAngle_0_1"]

# === Region definitions ===
in_non_gray = (delta_m > DeltaM_Low) & (delta_m < DeltaM_High)
in_gray = ~in_non_gray

# === Data selections ===
angle_non_gray = angle[in_non_gray]
angle_gray = angle[in_gray]

# === Histogram Plot ===
data = [angle_non_gray, angle_gray]
colors = ["#007C91", "#2E2E2E"]
labels = [
    r"uds ($0.16 < \Delta m_{\pi}(D_s^+ - D^0) < 0.27$ GeV/$c^2$)",
    r"uds (Outside peak region)"
]

plt.hist(
    data,
    bins=Bins,
    range=Range,
    stacked=Stacked,
    density=Density,
    histtype="step",
    linewidth=2,
    color=colors,
    label=labels
)

# === Titles ===
plt.title(r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc="right")

# === Axis Labels ===
plt.xlabel(r"$\theta (D^0 e^+)$ [rad]")
plt.ylabel(r"$Entries/(\;{:.3f}\;rad$)".format(perBin))
plt.legend()
plt.show()


In [ ]:
print(abs(DataFrames["All"][(abs(DataFrames["All"]['e_mcPDG'])==11) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=-1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']<=0.06) & (DataFrames["All"]['Ds_gammaveto_em_electronID']>=0.1)]['e_genMotherPDG']).value_counts(normalize=True,dropna=False).apply(lambda x: f"{x:.6f}"))

In [ ]:
Stacked = False
Density = True
Bins = 50
# var = 'Ds_diff_D0pi'
# var = 'Ds_massDifference_0'
var = 'e_pt'
Range = [0.0, 0.6]
dM = 0.06
FD = -1
BS = -1
eID = 0.1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$BDT \geq 0.531$'
label2= r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$'

labels=[label1,label2]
colors=["#A63F00", "#6A3D9A"]
data=[
      DataFrames[Samples][(DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=0.531) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames[Samples]['Ds_gammaveto_em_electronID']>=eID)][var],
      DataFrames[Samples][(DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames[Samples]['Ds_gammaveto_em_electronID']>=eID)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=2, range=Range)
# plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# plt.axvline(0.16,ls='--',color='gray')

# Title
#--------
plt.title(r'$D^{*0} \rightarrow D^{0} \; \pi^{0} / \gamma$ (Peak)', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m_{e}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 30000)
plt.legend()
plt.show()

In [ ]:
len(DataFrames[Samples][(abs(DataFrames[Samples]['Ds_mcPDG'])==423) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames[Samples]['Ds_gammaveto_em_electronID']>=eID)])

In [ ]:
len(DataFrames[Samples][(abs(DataFrames[Samples]['Ds_mcPDG'])==423) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames[Samples]['Ds_gammaveto_em_electronID']>=eID)])

In [ ]:
print(abs(DataFrames["Signal"][(np.cos(DataFrames["Signal"]["Ds_daughterAngle_0_1"])<=2)]['Ds_isSignal']).value_counts(normalize=False,dropna=False).apply(lambda x: f"{x:.6f}"))

In [ ]:
Stacked = True
Density = False
Bins = 50
var = 'Ds_diff_D0pi'
# i = 'Ds_massDifference_0'
Range = [0.1, 0.55]
dM = 0.06
FD = -1
BS = -1
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Other$'
label2= r'$D^{0}$'
label3= r'$D^{*0} \rightarrow D^{0} \; \pi^{0} / \gamma$'
label4= r'$D^{*+} \rightarrow D^{0} \; \pi^{+}$'

labels=[label1,label2,label3,label4]
colors=['C5','C4','C1','C2',]
data=[
      DataFrames[Samples][((abs(DataFrames[Samples]['D0_isSignal']).isna()) | ((abs(DataFrames[Samples]['D0_isSignal'])==0))) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# plt.axvline(0.16,ls='--',color='gray')

# Title
#--------
# plt.title(r'$BDT \geq 0.531$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 30000)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
var = 'Ds_diff_D0pi'
# i = 'Ds_massDifference_0'
Range = [0.1, 0.55]
BS = -1
dM = 0.06
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Other$'
label2= r'$D^{0}$'
label3= r'$D^{*0} \rightarrow D^{0} \; \pi^{0} / \gamma$'
label4= r'$D^{*+} \rightarrow D^{0} \; \pi^{+}$'

labels=[label1,label2,label3,label4]
colors=['C5','C4','C1','C2',]
data=[
      FakeD0_Truth[(FakeD0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (FakeD0_Truth['Ds_gammaveto_M_Correction']<=dM) & (FakeD0_Truth['Ds_gammaveto_em_electronID']>=eID)][var],
      D0_Truth[(D0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (D0_Truth['Ds_gammaveto_M_Correction']<=dM) & (D0_Truth['Ds_gammaveto_em_electronID']>=eID)][var],
      Dstar0_Truth[(abs(Dstar0_Truth['Ds_mcPDG'])!=423) & (Dstar0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (Dstar0_Truth['Ds_gammaveto_M_Correction']<=dM) & (Dstar0_Truth['Ds_gammaveto_em_electronID']>=eID)][var],
      DstarPlus_Truth[(abs(DstarPlus_Truth['Ds_mcPDG'])!=413) & (DstarPlus_Truth['Ds_extraInfo_BkgBDT']>=BS) & (DstarPlus_Truth['Ds_gammaveto_M_Correction']<=dM) & (DstarPlus_Truth['Ds_gammaveto_em_electronID']>=eID)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# plt.axvline(0.16,ls='--',color='gray')

# Title
#--------
plt.title(r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m_{\pi}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 6000)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = True
Bins = 50
var = 'Ds_Ds_starminusDs_M_Correction'
# i = 'Ds_massDifference_0'
Range = [0.0, 0.4]
BS = -1
dM = 0.06
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Other$'
label2= r'$D^{0}$'
label4= r'$D^{*+} \rightarrow D^{0} \; \pi^{+}$'

labels1=[label1,label2,label4]
colors1=['C5','C4','C2']
data1=[
      FakeD0_Truth[(FakeD0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (FakeD0_Truth['Ds_gammaveto_M_Correction']>=dM) & (FakeD0_Truth['Ds_gammaveto_em_electronID']>=0.1)][var],
      D0_Truth[(D0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (D0_Truth['Ds_gammaveto_M_Correction']>=dM) & (D0_Truth['Ds_gammaveto_em_electronID']>=0.1)][var],
      DstarPlus_Truth[(DstarPlus_Truth['Ds_extraInfo_BkgBDT']>=BS) & (DstarPlus_Truth['Ds_gammaveto_M_Correction']>=dM) & (DstarPlus_Truth['Ds_gammaveto_em_electronID']>=0.1)][var],
      ]
labels2=[r'$D^{*0} \; (Comb.)$',r'$D^{*0} \; (Peak)$']
colors2=['C1','C3']
data2=[
      Dstar0_Truth[(abs(Dstar0_Truth["Ds_mcPDG"])!=423) & (Dstar0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (Dstar0_Truth['Ds_gammaveto_M_Correction']>=dM) & (Dstar0_Truth['Ds_gammaveto_em_electronID']>=0.1)][var],
      Dstar0_Truth[(abs(Dstar0_Truth["Ds_mcPDG"])==423) & (Dstar0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (Dstar0_Truth['Ds_gammaveto_M_Correction']>=dM) & (Dstar0_Truth['Ds_gammaveto_em_electronID']>=0.1)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data1, color=colors1, label=labels1, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=2, range=Range)
plt.hist(data2, color=colors2, label=labels2, density=Density, stacked=True, bins=Bins, alpha=1, histtype='step', linewidth=2, range=Range)
# plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# plt.axvline(0.16,ls='--',color='gray')

# Title
#--------
# plt.title(r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m_{\pi}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 6000)
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# === USER OPTIONS ===
Density = False       # Normalize histograms
Bins = 50
Range = [0.1, 0.55]   # GeV
eID = 0.0             # Electron ID threshold
dM = 0.06             # m(ee) cut
variable = 'Ds_diff_D0pi'
perBin = ((Range[1] - Range[0]) / Bins) * 1000
print(f"Width Per Bin: {perBin:.2f} MeV")

# === COLORS & LABELS ===
sub_labels = [
    r'$e$',
    # r'$\pi^0 \rightarrow \gamma e^+e^-$',
]
sub_colors = ["#007C91"]  # dark cyan, blue, charcoal

# === MAIN CATEGORY CONFIGURATION ===
main_categories = [
    ("Other", FakeD0_Truth),
    (r"$D^0$", D0_Truth),
    (r"$D^{*0} \rightarrow D^0 \pi^0/\gamma$", Dstar0_Truth),
    (r"$D^{*+} \rightarrow D^0 \pi^+$", DstarPlus_Truth)
]

# === LOOP OVER MAIN CATEGORIES ===
for title_label, df in main_categories:
    fig, ax = plt.subplots(figsize=(6.5, 5))

    # Prepare the 3 subcategory histograms
    hists = []
    for mother_val in [11]:
        mask = (
            (df['Ds_gammaveto_em_electronID'] >= eID) &
            (df['Ds_gammaveto_M_Correction'] >= dM) &
            (abs(df['e_mcPDG']) == mother_val)
        )
        hists.append(df[mask][variable])

    # Plot all 3 subcategories
    ax.hist(hists,
            bins=Bins,
            range=Range,
            density=Density,
            histtype='step',
            linewidth=2,
            color=sub_colors,
            label=sub_labels)
    plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
    plt.axvline(0.16,ls='--',color='gray')
    plt.axvline(0.26,ls='--',color='gray')
    plt.axvspan(0.26,Range[1],color='gray',alpha=0.2)

    # Axis and title formatting
    ax.set_xlabel(r'$\Delta m_{\pi}(D_s^{+} - D^0)\;[\mathrm{GeV}/c^2]$')
    ax.set_ylabel(r'$Entries / ({:.2f}\; \mathrm{{MeV}}/c^2)$'.format(perBin))
    ax.set_title(title_label, loc='left', fontsize=11)
    ax.set_title(r'$\int\mathcal{L}dt \approx 1444$ fb$^{-1}$', loc='right')
    ax.legend(fontsize=10)
    # plt.tight_layout()
    plt.show()


In [ ]:
Stacked = False
Density = False
Bins = 50
var = 'Ds_diff_D0pi'
# i = 'Ds_massDifference_0'
Range = [0.1, 0.55]
BS = -1
dM = 0.06
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Other$'
label2= r'$D^{0}$'
label4= r'$D^{*+} \rightarrow D^{0} \; \pi^{+}$'

labels1=[label1,label2,label4]
colors1=['C5','C4','C2']
data1=[
      FakeD0_Truth[(FakeD0_Truth['e_genMotherPDG']==22) & (FakeD0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (FakeD0_Truth['Ds_gammaveto_M_Correction']<=dM) & (FakeD0_Truth['Ds_gammaveto_em_electronID']>=0.1)][var],
      D0_Truth[(D0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (D0_Truth['Ds_gammaveto_M_Correction']<=dM) & (D0_Truth['Ds_gammaveto_em_electronID']>=0.1)][var],
      DstarPlus_Truth[(abs(DstarPlus_Truth["Ds_mcPDG"])==413) & (DstarPlus_Truth['Ds_extraInfo_BkgBDT']>=BS) & (DstarPlus_Truth['Ds_gammaveto_M_Correction']<=dM) & (DstarPlus_Truth['Ds_gammaveto_em_electronID']>=0.1)][var],
      ]
labels2=[r'$D^{*0} \; (Comb.)$']
colors2=['C1']
data2=[
      Dstar0_Truth[(abs(Dstar0_Truth["e_genMotherPDG"])!=22) & (abs(Dstar0_Truth["Ds_mcPDG"])!=423) & (Dstar0_Truth['Ds_extraInfo_BkgBDT']>=BS) & (Dstar0_Truth['Ds_gammaveto_M_Correction']<=dM) & (Dstar0_Truth['Ds_gammaveto_em_electronID']>=0.1)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data1, color=colors1, label=labels1, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
plt.hist(data2, color=colors2, label=labels2, density=Density, stacked=True, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# plt.axvline(0.16,ls='--',color='gray')

# Title
#--------
plt.title(r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m_{\pi}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 6000)
plt.legend()
plt.show()

In [ ]:
# Stacked = False
# Density = False
# Bins = 50
# Range = [0.1, 0.55]
# Op = 0.531
# dM = -1
# # i = 'e_cos_theta'
# # i = 'Ds_vpho_CMS_daughterAngle'
# i = 'Ds_diff_D0pi'
# # i = 'Ds_chiProb_noIP'
# # i = 'Ds_chiProb'
# # i = 'Ds_extraInfo_FastBDT'
# # i = 'D0_chiProb'
# # i = 'Ds_Ds_starminusDs_M_Correction'
# # i = "Ds_gammaveto_M_Correction"
# # i = 'D0_chiProb'
# # i = "Ds_L_diff"
# # var = 'e_cos_theta'
# # i = 'e_pt'
# perBin = ((Range[1] - Range[0])/Bins)*1000
# # perBin = ((Range[1] - Range[0])/Bins)
# print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# # label3= r'$D^{0}$'
# # label4= r'$Other$'

# # labels1=[label3,label4]
# # colors1=['C1','C2']
# # data1=[
# #       DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_NoDstarplusDstar0'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
# #       DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Other'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
# #       ]
# labels2=[r'$D^{*0} \; (Comb.)$',r'$D^{*0} \; (Peak)$']
# colors2=['C1','C2']
# data2=[
#       DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])!=423) & (abs(DataFrames["All"]['Ds_D0_Dstar0'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
#       DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])==423) & (abs(DataFrames["All"]['Ds_D0_Dstar0'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
#       ]
# labels3=[r'$D^{*+} \; (Comb.)$',r'$D^{*+} \; (Peak)$']
# colors3=['C3','C4']
# data3=[
#       DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])!=413) & (abs(DataFrames["All"]['Ds_D0_Dstarplus'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
#       DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])==413) & (abs(DataFrames["All"]['Ds_D0_Dstarplus'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
#       ]

# # factor = 0.1
# # plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]["Ds_extraInfo_BkgBDT"]>=Op)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]["Ds_extraInfo_BkgBDT"]>=Op)][i]), ls='--', linewidth=1.5)
# # plt.hist(data1, color=colors1, label=labels1, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.hist(data2, color=colors2, label=labels2, density=Density, stacked=True, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.hist(data3, color=colors3, label=labels3, density=Density, stacked=True, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# # plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# # plt.axvline(0.58,ls='--',color='gray')

# # Title
# #--------
# # plt.title(r'$BDT \; \geq 0.531$', loc = "left")
# # Label
# #-------
# # plt.ylabel(r'$Entries/(\; {width:.2f}\;)$'.format(width = perBin))
# # plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c)$'.format(width = perBin))
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# # plt.xlabel(r'$p_{t} (e^{+}) [GeV/c]$')
# # plt.xlabel(r'$\Delta \theta(D_s^{+} \; K^{+/-}/K_{s}^{0}) \; [rad]$')
# # plt.xlabel(r'$cos\theta \; (e^{+})$')
# # plt.xlabel(r'$p-value(D^{0})$')
# # plt.xlabel(r'$p-value(D_{s}^{+})$')
# # plt.xlabel(r'$p-value_{IP}(D_{s}^{+})$')
# # plt.xlabel(r'$Fake D^{0} Suppression(D^{0})$')
# # plt.xlabel(r'$m(e_{sig}^{+} e_{ROE}^{-})\;[GeV/c^{2}]$')
# # plt.xlabel(r'$p_{t} \; (e^{+})\;[GeV/c]$')
# plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# # plt.xlabel(r'$\Delta m(D_s^{*+} - D_{s}^{+})\;[GeV/c^{2}]$')
# # plt.xlabel(r'$cos\theta \; (e^{+})$')
# # plt.xlabel(r'$p-value(D^{0})$')
# # plt.xlabel(r'$\mid \vec{x}_{D_{s}^{+}} - \vec{x}_{D^{0}} \mid \; [cm]$')
# # plt.xlabel(r'$dz \; (e^{+}) \; [cm]$')
# # plt.yscale("log")
# # plt.xscale("log")
# plt.ylim(0, 8000)
# plt.legend()
# plt.show()

In [ ]:
# Combined = pd.concat([DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])!=413) & (abs(DataFrames["All"]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames["All"]['D0_isSignal'])==1) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=BS)],
#                       DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])!=423) & (abs(DataFrames["All"]['Ds_D0_Dstar0'])==1) & (abs(DataFrames["All"]['D0_isSignal'])==1) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=BS)],
#                       DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames["All"]['D0_isSignal'])==1) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=BS)],
#                       DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Other'])==1) | ((abs(DataFrames["All"]['D0_mcPDG'])==421) & (abs(DataFrames["All"]['D0_isSignal'])==0)) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=BS)],
#                       ], ignore_index=True, sort=False)

In [ ]:
# Stacked = False
# Density = True
# Bins = 50
# Range = [0.1, 0.55]
# Op = 0.531
# dM = -1
# # i = 'e_cos_theta'
# # i = 'Ds_vpho_CMS_daughterAngle'
# i = 'Ds_diff_D0pi'
# # i = 'Ds_chiProb_noIP'
# # i = 'Ds_chiProb'
# # i = 'Ds_extraInfo_FastBDT'
# # i = 'D0_chiProb'
# # i = 'Ds_Ds_starminusDs_M_Correction'
# # i = "Ds_gammaveto_M_Correction"
# # i = 'D0_chiProb'
# # i = "Ds_L_diff"
# # var = 'e_cos_theta'
# # i = 'e_pt'
# perBin = ((Range[1] - Range[0])/Bins)*1000
# # perBin = ((Range[1] - Range[0])/Bins)
# print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# label1= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'
# label2= r'$D^{*0} \rightarrow D^{0} \gamma, D^{0} \pi^{0}$'
# label3= r'$D^{0}$'
# label4= r'$Other$'

# labels=[label1,label2,label3,label4]
# colors=['C2','C1','C4','C5']
# data=[
#       DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])!=413) & (abs(DataFrames["All"]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames["All"]['D0_isSignal'])==1) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=BS)][var],
#       DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])!=423) & (abs(DataFrames["All"]['Ds_D0_Dstar0'])==1) & (abs(DataFrames["All"]['D0_isSignal'])==1) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=BS)][var],
#       DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames["All"]['D0_isSignal'])==1) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=BS)][var],
#       DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Other'])==1) | ((abs(DataFrames["All"]['D0_mcPDG'])==421) & (abs(DataFrames["All"]['D0_isSignal'])==0)) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=BS)][var],
#       ]

# # factor = 0.1
# # plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]["Ds_extraInfo_BkgBDT"]>=Op)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]["Ds_extraInfo_BkgBDT"]>=Op)][i]), ls='--', linewidth=1.5)
# plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.hist(Combined[i], label="Combined", density=Density, bins=Bins, alpha=1, histtype='step', linewidth=1.5, ls='--', range=Range)
# # plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# # plt.axvline(0.58,ls='--',color='gray')

# # Title
# #--------
# # plt.title(r'$BDT \; \geq 0.531$', loc = "left")
# # Label
# #-------
# # plt.ylabel(r'$Entries/(\; {width:.2f}\;)$'.format(width = perBin))
# # plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c)$'.format(width = perBin))
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# # plt.xlabel(r'$p_{t} (e^{+}) [GeV/c]$')
# # plt.xlabel(r'$\Delta \theta(D_s^{+} \; K^{+/-}/K_{s}^{0}) \; [rad]$')
# # plt.xlabel(r'$cos\theta \; (e^{+})$')
# # plt.xlabel(r'$p-value(D^{0})$')
# # plt.xlabel(r'$p-value(D_{s}^{+})$')
# # plt.xlabel(r'$p-value_{IP}(D_{s}^{+})$')
# # plt.xlabel(r'$Fake D^{0} Suppression(D^{0})$')
# # plt.xlabel(r'$m(e_{sig}^{+} e_{ROE}^{-})\;[GeV/c^{2}]$')
# # plt.xlabel(r'$p_{t} \; (e^{+})\;[GeV/c]$')
# plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# # plt.xlabel(r'$\Delta m(D_s^{*+} - D_{s}^{+})\;[GeV/c^{2}]$')
# # plt.xlabel(r'$cos\theta \; (e^{+})$')
# # plt.xlabel(r'$p-value(D^{0})$')
# # plt.xlabel(r'$\mid \vec{x}_{D_{s}^{+}} - \vec{x}_{D^{0}} \mid \; [cm]$')
# # plt.xlabel(r'$dz \; (e^{+}) \; [cm]$')
# # plt.yscale("log")
# # plt.xscale("log")
# # plt.ylim(0, 8000)
# plt.legend()
# plt.show()

In [ ]:
# Stacked = True
# Density = False
# Bins = 50
# i = 'Ds_diff_D0pi'
# # i = 'Ds_massDifference_0'
# Range = [0.1, 0.55]
# BD = 0.531
# perBin = ((Range[1] - Range[0])/Bins)*1000
# print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# label1= r'$c \bar{c}$'
# label2= r'$BB$'
# label3= r'$u \bar{u}, \; d \bar{d}, \;s \bar{s}$'
# label4= r'$\tau^{+} \tau^{-}$'

# labels=[label1,label2,label3,label4]
# # colors=['C2','C1','C3','C4','C5']
# data=[DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_extraInfo_BkgBDT']>=BD)][i],
#       DataFrames["BB"][(DataFrames["BB"]['Ds_extraInfo_BkgBDT']>=BD)][i],
#       DataFrames["uds"][(DataFrames["uds"]['Ds_extraInfo_BkgBDT']>=BD)][i],
#       DataFrames["taupair"][(DataFrames["taupair"]['Ds_extraInfo_BkgBDT']>=BD)][i],
#       ]

# factor = 0.5
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BD)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BD)][i]), ls='--', linewidth=1.5)
# plt.hist(data[::-1], label=labels[::-1], density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# # plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# # plt.axvline(0.15,ls='--',color='gray')

# # Title
# #--------
# # plt.title(r'$BCS: Vertex \; Fit \; \chi^{2}$', loc = "left")
# plt.title(r'$D_s^{+} \rightarrow [D^{0} \rightarrow K^{-} \pi^{+}] e^{+} \nu_{e}$', loc = "left")
# # Label
# #-------
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# # plt.yscale("log")
# # plt.xscale("log")
# plt.ylim(0, 25000)
# plt.legend()
# plt.show()

In [ ]:
# from Functions import get_df_sig_bkg,get_nsig_nbkg,plot_var_sig_bkg,optimize_SB,plot_save
# # "Ds_extraInfo_BkgBDT"
# # "Ds_BS"
# optimize_SB(df_sig=DataFrames["Signal"],
#             df_bkg=DataFrames["All"],
#             var="Ds_extraInfo_BkgBDT",
#             Signal=DataFrames["Signal"],
#             Background=DataFrames["All"],
#             FoM="Ds_extraInfo_BkgBDT",
#             xlabel='Classifier Output',
#             Bins=50,Range=[0,1],
#             varmin=0,varmax=0.93,select='right',Width=False)

# Save BDT Output

In [ ]:
# # Define histogram settings
# nbins = 100  # Adjust as needed
# xmin, xmax = 0.135, 0.18  # Define the bin range appropriately

# # Open a ROOT file to store histograms
# output_file = uproot.recreate("/home/belle2/amubarak/C03-Grid/TopoAna_Binned.root")

# # List of DataFrames explicitly defined
# all_dataframes = {"Dstarplus": DstarPlus_TopoAna, 
#                   "Dstar0": Dstar0_TopoAna, 
#                   "D0": D0_TopoAna, 
#                   "FakeD0": FakeD0_TopoAna, 
#                   "Signal": DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=0.531)]}

# # Loop through the DataFrames and create histograms
# for sample, df in all_dataframes.items():
#     if 'Ds_diff_D0pi' in df.columns:
#         # Create histogram using numpy
#         hist = np.histogram(df['Ds_diff_D0pi'].dropna(), bins=nbins, range=(xmin, xmax))
        
#         # Store histogram in ROOT format using uproot as TH1F
#         output_file[f"hist_{sample}"] = hist

# # Save and close the file
# output_file.close()
# print("Histograms saved in TH1F format to output_histograms.root")

In [ ]:
# # store the dataframe with the sweight columns in a new file
# with uproot.recreate('/home/belle2/amubarak/C03-Grid/TopoAna.root') as f:
#     f['Dstarplus'] = DstarPlus_TopoAna
#     f['Dstar0'] = Dstar0_TopoAna
#     f['D0'] = D0_TopoAna
#     f['FakeD0'] = FakeD0_TopoAna

In [ ]:
# len(DataFrames[samples[i]]) - len(DstarPlus_TopoAna) - len(Dstar0_TopoAna) - len(D0_TopoAna) - len(FakeD0_TopoAna)

In [ ]:
# len(DataFrames[samples[i]]) - len(DataFrames[samples[i]][(abs(DataFrames[samples[i]]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames[samples[i]]['D0_isSignal'])==1)]) - len(DataFrames[samples[i]][(abs(DataFrames[samples[i]]['Ds_D0_Dstar0'])==1) & (abs(DataFrames[samples[i]]['D0_isSignal'])==1)]) - len(DataFrames[samples[i]][(abs(DataFrames[samples[i]]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames[samples[i]]['D0_isSignal'])==1)]) - len(DataFrames[samples[i]][(abs(DataFrames[samples[i]]['Ds_D0_Other'])==1) | ((abs(DataFrames[samples[i]]['D0_mcPDG'])==421) & (abs(DataFrames[samples[i]]['D0_isSignal'])==0))])

In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

# # === Filter to true electrons only ===
# df = DataFrames["uds"][
#     (DataFrames["uds"]["Ds_gammaveto_M_Correction"] <= 0.06)
# ]

# # === invmass function (your definition) ===
# def invmass(m1, m, p1, p2, px1, px2, py1, py2, pz1, pz2):
#     if m == 'pi':
#         m2 = 0.13957039
#     elif m == 'e':
#         m2 = 0.00051099895
#     elif m == 'K':
#         m2 = 0.493677
#     elif m == 'p':
#         m2 = 0.938272
#     else:
#         raise ValueError("Unsupported mass hypothesis.")

#     E1 = np.sqrt(m1**2 + p1**2)
#     E2 = np.sqrt(m2**2 + p2**2)

#     P1 = np.array([ px1, py1, pz1])
#     P2 = np.array([ px2, py2, pz2])

#     P = p1**2 + p2**2 + (2*(np.dot(P1,P2)))
    
#     return np.sqrt((E1+E2)**2 - P)

# # === Compute delta m with different mass hypotheses
# df['DeltaM_electron'] = df.apply(lambda row: invmass(row['D0_M'], 'e', row['D0_p'], row['e_p'], 
#                                              row['D0_px'], row['e_px'], row['D0_py'], row['e_py'], row['D0_pz'], row['e_pz']) - row['D0_M'], axis=1)
# df['DeltaM_pion'] = df.apply(lambda row: invmass(row['D0_M'], 'pi', row['D0_p'], row['e_p'], 
#                                         row['D0_px'], row['e_px'], row['D0_py'], row['e_py'], row['D0_pz'], row['e_pz']) - row['D0_M'], axis=1)
# df['DeltaM_kaon'] = df.apply(lambda row: invmass(row['D0_M'], 'K', row['D0_p'], row['e_p'], 
#                                                  row['D0_px'], row['e_px'], row['D0_py'], row['e_py'], row['D0_pz'], row['e_pz']) - row['D0_M'], axis=1)
# angle_rad = df["Ds_daughterAngle_0_1"]

# # === Build correlation matrix
# cor_df = pd.DataFrame({
#     r"$\Delta m_e$ [GeV/$c^2$]": df['DeltaM_electron'],
#     r"$\Delta m_\pi$ [GeV/$c^2$]": df['DeltaM_pion'],
#     r"$\Delta m_K$ [GeV/$c^2$]": df['DeltaM_kaon'],
# #     r"$\Delta m_p$ [GeV/$c^2$]": dm_p,
#     r"$\theta(D^0, e^+)$ [rad]": angle_rad,
#     r"$p(e^+)$ [GeV/$c$]": df['e_p']
# })

# corr_matrix = cor_df.corr(method="pearson")

# # === Plot heatmap
# plt.figure(figsize=(8, 7))
# im = plt.imshow(corr_matrix, cmap="coolwarm", vmin=-1, vmax=1)

# plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=45, ha='right')
# plt.yticks(range(len(corr_matrix.index)), corr_matrix.index)

# for i in range(len(corr_matrix.columns)):
#     for j in range(len(corr_matrix.index)):
#         val = corr_matrix.iloc[j, i]
#         plt.text(i, j, f"{val:.2f}", ha='center', va='center', color='black')

# plt.colorbar(im, label="Pearson Correlation")
# plt.title("Correlation Matrix Using Your invmass() (uds true electrons)")
# # plt.tight_layout()
# plt.show()